# Importing required packages:

In [ ]:
import nltk
import numpy as np
import scipy
import javalang
import re
import sys
import pyparsing
import json
import random
import pandas as pd
import matplotlib.pyplot as plt
import pickle
from pandas import DataFrame

import os
from os import listdir
from os.path import isfile, join, splitext,split
import csv
import math
import time
import warnings

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn import metrics
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import confusion_matrix, roc_curve, roc_auc_score

from sklearn.feature_selection import RFE, VarianceThreshold, SelectKBest, f_classif, mutual_info_classif, chi2, f_regression, SelectFpr, SelectFdr
from sklearn.calibration import CalibratedClassifierCV

from sklearn.ensemble import RandomForestClassifier

from sklearn.neighbors import KNeighborsClassifier

from sklearn.naive_bayes import GaussianNB

from sklearn.svm import SVR

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, Flatten, Conv1D, Conv2D, MaxPooling1D, MaxPooling2D, Dropout, Input, concatenate
from tensorflow.keras.models import Model

In [ ]:
from nltk import bigrams, trigrams
from collections import Counter, defaultdict
from anytree import Node, RenderTree, PreOrderIter, PostOrderIter, LevelOrderIter
from anytree.exporter import DotExporter
import graphviz

In [ ]:
warnings.filterwarnings("ignore")

In [ ]:
threshold = 0.5130

# Loading the data:

In [ ]:
with open("./data_2/d/d_1_total.txt","rb") as fp:
    train_1 = pickle.load(fp)
with open("./data_2/d/d_2_total.txt","rb") as fp:
    train_2 = pickle.load(fp)

## ngram_type:
1: word ngrams

2: structural ngrams

In [ ]:
ngram_type = 1

In [ ]:
AST_type = 0

## Defining methods to count number of occurances of unigrams and bigrams:

In [ ]:
np.set_printoptions(threshold=sys.maxsize)

def comment_remover(a):
    a = re.sub(re.compile("//.*?\n"),"",a)
    a = re.sub(re.compile("/\*.*?\*/",re.DOTALL),"",a)
    a = re.sub(re.compile("/\*.*?\n",re.DOTALL),"",a)
    a = re.sub(re.compile("\*.*?\n",re.DOTALL),"",a)
    a = re.sub(re.compile("/\*.*?$",re.DOTALL),"",a)
    return(a)

def preprocess(file_text):
    file_text = comment_remover(file_text)
    file_text_2 = file_text
    file_text_2 = re.sub(r'["](.*?)["]','String_1',file_text_2)
    
    file_text_2 = re.sub(r'[\'](.*?)[\']','Literal_1',file_text_2)
    
    for a in ['\\','#','"@@"','"','\'']:
        file_text_2 = file_text_2.replace(a,'')
    file_text_2 = file_text_2.replace('`','')
    return file_text_2

def token_list(file_text):
    tokens_list = javalang.tokenizer.tokenize(file_text)
    tokens = []
    for i in tokens_list:
        class_name = i.__class__.__name__
        value = i.value
        if (value in ('Override','start_sentence','String_1','Literal_1')):
            tokens.append(value)
        elif (value in (';')):
            tokens.append('end_sentence')
        elif (value in ('.')):
            tokens.append('dot')
        elif (value in (',')):
            tokens.append('comma')
        elif (value in ('[')):
            tokens.append('open_square_brackets')
        elif (value in (']')):
            tokens.append('closed_square_brackets')
        elif (value in ('(','{')):
            tokens.append('open_parenthesis')
        elif (value in (')','}')):
            tokens.append('closed_parenthesis')
        elif (value in ('=','+=','-=','*=','/=','%=','<<=','>>=','&=','|=','^=')):
            tokens.append('assignment_operator')
        elif (value in ('+','-','*','/','%','++','--')):
            tokens.append('arithmetic_operator')
        elif (value in ('==','!=','>','<','>=','<=')):
            tokens.append('relational_operator')
        elif (value in ('&','|','^','~','<<','>>','>>>')):
            tokens.append('bitwise_operator')
        elif (value in ('&&','||','!')):
            tokens.append('logical_operator')
        elif (class_name in ('Identifier')):
            if (len(value) == 1):
                tokens.append('Identifier_1')
            else:
                tokens.append('Identifier_2')
        elif ((class_name in ['Integer','DecimalInteger','FloatingPoint','DecimalFloatingPoint']) and (value[-1] not in ['L','l','e','E','f','F','d','D'])):
                value_1 = float(value)
                if (value_1 == 0):
                    tokens.append('zero_number')
                else:
                    tokens.append('non_zero_number')
        elif (class_name in class_types):
            tokens.append(class_name)
        elif (value in ('@')):
            continue
        else:
            tokens.append(value)
    return tokens

def compute_counts(tokens):
    for i in range(len(tokens)):
        word_existed = 0
        for j in range(len(words)):
            if (tokens[i] == words[j]):
                counts[j]+=1
                word_existed = 1
                break
        if (word_existed == 0):
            counts[n-1]+=1
    return counts

def compute_counts_2d(tokens):
    for i in range(len(tokens)-1):
        i_1 = -1
        i_2 = -1
        for j in range(len(words)):
            if (tokens[i] == words[j]):
                i_1 = j
            if (tokens[i+1] == words[j]):
                i_2 = j
        if (i_1 == -1):
            i_1 = n-1
        if (i_2 == -1):
            i_2 = n-1
        counts_2d[i_1,i_2] += 1
    return counts_2d

def compute_counts_3d(tokens):
    for i in range(len(tokens)-2):
        i_1 = -1
        i_2 = -1
        i_3 = -1
        for j in range(len(words)):
            if (tokens[i] == words[j]):
                i_1 = j
            if (tokens[i+1] == words[j]):
                i_2 = j
            if (tokens[i+2] == words[j]):
                i_3 = j
        if (i_1 == -1):
            i_1 = n-1
        if (i_2 == -1):
            i_2 = n-1
        if (i_3 == -1):
            i_3 = n-1
        counts_3d[i_1,i_2,i_3] += 1
    return counts_3d

def compute_collective_counts(file):
    l = len(file)
    for i in range(l):
        if True:
            print('i:',i)
            file_text = file[i]["codes"]
            if (AST_type == 0 and ngram_type == 1):
                file_text_2 = preprocess(file_text)
                tokens = token_list(file_text_2)
            else:
                tree = javalang.parse.parse(file_text)
                i_2 = 0
                print_mode = 0
                path_1 = []
                tree_2 = create_tree(tree,i_2,path_1,print_mode)
                if (AST_type == 1):
                    tokens = [n.name for n in LevelOrderIter(tree_2)]
                elif (AST_type == 2):
                    tokens = [n.name for n in PreOrderIter(tree_2)]
                elif (AST_type == 3):
                    tokens = [n.name for n in PostOrderIter(tree_2)]
            counts_array = compute_counts(tokens)

            counts_array_2d = compute_counts_2d(tokens)
            counts_array_3d = compute_counts_3d(tokens)
    return(counts_array,counts_array_2d,counts_array_3d)

def compute_unigram(counts):
    # Add-one smoothing
    smoothed_counts = counts + 1
    unigram = []
    unigram_array = []
    for i in range(len(counts)):
        probability = smoothed_counts[i]/(np.sum(counts)+len(words))
        unigram.append([words[i],probability])
        unigram_array.append(probability)
    unigram = np.array(unigram)
    unigram_array = np.array(unigram_array)
    return unigram_array

def compute_unigram_without_smoothing(counts):
    unigram = []
    unigram_array = []
    for i in range(len(counts)):
        probability = counts[i]/(np.sum(counts))
        unigram.append([words[i],probability])
        unigram_array.append(probability)
    unigram = np.array(unigram)
    unigram_array = np.array(unigram_array)
    return unigram_array

def compute_bigram(counts_2d,counts_1):
    n = len(words)
    bigram = np.zeros((n,n))
    for i in range(len(counts_2d)):
        for j in range(len(counts_2d)):
            bigram[i,j] = (counts_2d[i,j]+1)/(counts_1[i]+len(words))
    return bigram

def compute_bigram_without_smoothing(counts_2d,counts_1):
    n = len(words)
    bigram = np.zeros((n,n))
    for i in range(len(counts_2d)):
        for j in range(len(counts_2d)):
            if (counts_1[i] != 0):
                bigram[i,j] = counts_2d[i,j]/counts_1[i]
            else:
                bigram[i,j] = 0
    return bigram

def compute_trigram_without_smoothing(counts_3d,counts_2d):
    n = len(words)
    trigram = np.zeros((n,n,n))
    for i in range(len(counts_3d)):
        for j in range(len(counts_3d)):
            for k in range(len(counts_3d)):
                if (counts_2d[i,j] != 0):
                    trigram[i,j,k] = counts_3d[i,j,k]/counts_2d[i,j]
                else:
                    trigram[i,j,k] = 0
    return trigram

def compute_bigram_without_smoothing_d(counts_2d,counts_1):
    d_bigram = np.zeros((n,n))
    for element in dense_bigram_1:
        i = int(element[0])
        j = int(element[1])
    
        if (counts_1[i] != 0):
            d_bigram[i,j] = counts_2d[i,j]/counts_1[i]
        else:
            d_bigram[i,j] = 0
    return d_bigram

def compute_trigram_without_smoothing_d(counts_3d,counts_2d):
    d_trigram = np.zeros((n,n,n))
    for element in dense_trigram_1:
        i = int(element[0])
        j = int(element[1])
        k = int(element[2])
    
        if (counts_2d[i,j] != 0):
            d_trigram[i,j,k] = counts_3d[i,j,k]/counts_2d[i,j]
        else:
            d_trigram[i,j,k] = 0
    return d_trigram

In [ ]:
def children_list(node):
    class_name = node.__class__
    children_list_1 = super(class_name,node).children
    return children_list_1
    
def print_line(file_text_1,position_line):
    lines_to_print = range(position_line,position_line+1)
    file_lines = file_text_1.splitlines()
    print('Line of code:')
    for a in lines_to_print:
        print(file_lines[a-1])
    return

def append_words(paths_1):
    for p in paths_1:
        for token in p:
            if token not in words:
                words.append(token)

def token_list_2(t):
    p = []
    p_class = []
    for path, node in t:
        if node not in p:
            p.append(node)
            p_class.append(node.__class__.__name__)
    return p_class

def bigram_list(tree,i,path_1,print_mode):
    path = path_1.copy()
    c = tree.__class__.__name__
    if (c not in ['NoneType','str','list','set','bool'] and i != 0):
        a_class = path[len(path)-1]
        bigrams.append([a_class,c])
    if (c not in ['NoneType','list']):
        path.append(c)
        i += 1
    if (c not in ['NoneType','str','list','set','bool']):
        dict_c = tree.__dict__
        if (c != 'CompilationUnit' and print_mode == 1):
            print('\n'*5,'Class name:',c)
            for j in dict_c:
                print(j,':')
                print(dict_c[j])
                if (j == '_position'):
                    position_line = dict_c[j].line
                    position_column = dict_c[j].column
                    print_line(file_text,position_line)
                print('\n')
        nodes = children_list(tree)
        for node in nodes:
            bigram_list(node,i,path,print_mode)
    else:
        if (c in ['list','set']):
            for element in tree:
                bigram_list(element,i,path,print_mode)
                
def trigram_list(tree,i,path_1,print_mode):
    path = path_1.copy()
    c = tree.__class__.__name__
    if (c not in ['NoneType','str','list','set','bool'] and i > 1):
        a_class = path[len(path)-2]
        b_class = path[len(path)-1]
        trigrams.append([a_class,b_class,c])
    if (c not in ['NoneType','list']):
        path.append(c)
        i += 1
    if (c not in ['NoneType','str','list','set','bool']):
        dict_c = tree.__dict__
        if (c != 'CompilationUnit' and print_mode == 1):
            print('\n'*5,'Class name:',c)
            for j in dict_c:
                print(j,':')
                print(dict_c[j])
                if (j == '_position'):
                    position_line = dict_c[j].line
                    position_column = dict_c[j].column
                    print_line(file_text,position_line)
                print('\n')
        nodes = children_list(tree)
        for node in nodes:
            trigram_list(node,i,path,print_mode)
    else:
        if (c in ['list','set']):
            for element in tree:
                trigram_list(element,i,path,print_mode)
                
def compute_counts_2d_s(b):
    for i in range(len(b)):
        i_1 = -1
        i_2 = -1
        for j in range(len(words)):
            if (b[i][0] == words[j]):
                i_1 = j
            if (b[i][1] == words[j]):
                i_2 = j
        if (i_1 == -1):
            i_1 = n-1
        if (i_2 == -1):
            i_2 = n-1
        counts_2d[i_1,i_2] += 1
    return counts_2d

def compute_counts_3d_s(t):
    for i in range(len(t)):
        i_1 = -1
        i_2 = -1
        i_3 = -1
        for j in range(len(words)):
            if (t[i][0] == words[j]):
                i_1 = j
            if (t[i][1] == words[j]):
                i_2 = j
            if (t[i][2] == words[j]):
                i_3 = j
        if (i_1 == -1):
            i_1 = n-1
        if (i_2 == -1):
            i_2 = n-1
        if (i_3 == -1):
            i_3 = n-1
        counts_3d[i_1,i_2,i_3] += 1
    return counts_3d

def compute_bigram_without_smoothing_s(counts_2d):
    n = len(words)
    bigram = np.zeros((n,n))
    for i in range(len(counts_2d)):
        for j in range(len(counts_2d)):
            a = sum(counts_2d[i])
            if (a != 0):
                bigram[i,j] = counts_2d[i,j]/a
            else:
                bigram[i,j] = 0
    return bigram

def compute_trigram_without_smoothing_s(counts_3d):
    n = len(words)
    trigram = np.zeros((n,n,n))
    for i in range(len(counts_3d)):
        for j in range(len(counts_3d)):
            for k in range(len(counts_3d)):
                a = sum(counts_3d[i,j])
                if (a != 0):
                    trigram[i,j,k] = counts_3d[i,j,k]/a
                else:
                    trigram[i,j,k] = 0
    return trigram

def parse_m(s):
    if not s.endswith(';'):
        s = s + ';'
    tokens = javalang.tokenizer.tokenize(s)
    parser = javalang.parser.Parser(tokens)
    m_tree = parser.parse_member_declaration()
    return m_tree

# Defining AST travesing methods:

In [ ]:
def create_tree(tree,i,path_1,print_mode):
    path = path_1.copy()
    c = tree.__class__.__name__
    if (i == 0):
        node_1 = Node(c)
    if (c not in ['NoneType','str','list','set','bool'] and i != 0):
        a_class = path[len(path)-1]
        node_1 = Node(c, parent=a_class)
    if (c not in ['NoneType','str','list','set','bool']):
        path.append(node_1)
        i += 1
    if (c not in ['NoneType','str','list','set','bool']):
        dict_c = tree.__dict__
        if (c != 'CompilationUnit' and print_mode == 1):
            print('\n'*5,'Class name:',c)
            for j in dict_c:
                print(j,':')
                print(dict_c[j])
                if (j == '_position'):
                    position_line = dict_c[j].line
                    position_column = dict_c[j].column
                    print_line(file_text,position_line)
                print('\n')
        nodes = children_list(tree)
        for node in nodes:
            create_tree(node,i,path,print_mode)
    else:
        if (c in ['list','set']):
            for element in tree:
                create_tree(element,i,path,print_mode)
    
    if (c not in ['NoneType','str','list','set','bool'] and i == 1):
        return node_1    

def children_list(node):
    class_name = node.__class__
    children_list_1 = super(class_name,node).children
    return children_list_1
 
def print_tree(tree_1):
#     print(RenderTree(tree_1))
    tree_2 = RenderTree(tree_1)
    for a_1,a_2,a_3 in tree_2:
        print("%s%s" % (a_1,a_3.name))

# Definig the list of words:

In [ ]:
ngram_type = 1
words = []
if (ngram_type == 1):
    class_types = ['Identifier_1','Identifier_2','zero_number','non_zero_number','Literal_1','Character','String_1','Boolean']
    word_types = ['end_sentence','dot','comma','open_square_brackets','closed_square_brackets','open_parenthesis','closed_parenthesis','assignment_operator','arithmetic_operator','relational_operator','bitwise_operator','logical_operator','Override']
    words_file = open('words.txt').read()
    for i in word_types:
        words.append(i)
    for i in class_types:
        words.append(i)
    for i in words_file.split():
        words.append(i)
    words.append('other')

elif (ngram_type == 2):
    with open("./data_2/words.txt","rb") as fp:
        words = pickle.load(fp)

n = len(words)
print('words:\n',words)
print('\nnumber of words:',n)

counts = np.zeros((n))
counts_2d = np.zeros((n,n))

## Computing collective counts (ngram_type = 1 or serial AST):

In [ ]:
ngram_type = 1

In [ ]:
# AST type => 0:words or parallel   1:BFS   2:pre-order   3:post-order
AST_type = 0

In [ ]:
n = len(words)
counts = np.zeros((n))
counts_2d = np.zeros((n,n))
counts_3d = np.zeros((n,n,n))
calculated_values = compute_collective_counts(train_1)
counts_array_1 = calculated_values[0]
counts_array_2d_1 = calculated_values[1]
counts_array_3d_1 = calculated_values[2]
print('words list:\n',words,'\n')
print(counts_array_1)
print(counts_array_2d_1)

In [ ]:
counts = np.zeros((n))
counts_2d = np.zeros((n,n))
counts_3d = np.zeros((n,n,n))
calculated_values = compute_collective_counts(train_2)
counts_array_2 = calculated_values[0]
counts_array_2d_2 = calculated_values[1]
counts_array_3d_2 = calculated_values[2]
print('words list:\n',words,'\n')
print(counts_array_2)
print(counts_array_2d_2)

In [ ]:
unigram_1 = compute_unigram_without_smoothing(counts_array_1)

unigram_2 = compute_unigram_without_smoothing(counts_array_2)

bigram_1 = compute_bigram_without_smoothing(counts_array_2d_1,counts_array_1)

bigram_2 = compute_bigram_without_smoothing(counts_array_2d_2,counts_array_2)

print('words list:\n',words,'\n')
trigram_1 = compute_trigram_without_smoothing(counts_array_3d_1,counts_array_2d_1)
print('trigram_1:\n',trigram_1[4])

print('words list:\n',words,'\n')
trigram_2 = compute_trigram_without_smoothing(counts_array_3d_2,counts_array_2d_2)
print('trigram_2:\n',trigram_2[4])

In [ ]:
# files
np.save('./data_2/unigram_1.npy',unigram_1)
np.save('./data_2/unigram_2.npy',unigram_2)

np.save('./data_2/bigram_1.npy',bigram_1)
np.save('./data_2/bigram_2.npy',bigram_2)

np.save('./data_2/trigram_1.npy',trigram_1)
np.save('./data_2/trigram_2.npy',trigram_2)

In [ ]:
# methods
np.save('./data_2/unigram_1_m.npy',unigram_1)
np.save('./data_2/unigram_2_m.npy',unigram_2)

np.save('./data_2/bigram_1_m.npy',bigram_1)
np.save('./data_2/bigram_2_m.npy',bigram_2)

In [ ]:
# files
unigram_1 = np.load('./data_2/unigram_1.npy')
unigram_2 = np.load('./data_2/unigram_2.npy')

bigram_1 = np.load('./data_2/bigram_1.npy')
bigram_2 = np.load('./data_2/bigram_2.npy')

trigram_1 = np.load('./data_2/trigram_1.npy')
trigram_2 = np.load('./data_2/trigram_2.npy')

In [ ]:
# methods
unigram_1 = np.load('./data_2/unigram_1_m.npy')
unigram_2 = np.load('./data_2/unigram_2_m.npy')

bigram_1 = np.load('./data_2/bigram_1_m.npy')
bigram_2 = np.load('./data_2/bigram_2_m.npy')

## Computing collective counts (ngram_type = 2):

In [ ]:
ngram_type = 2

In [ ]:
# files
n = len(words)
length = len(train_1)
counts = np.zeros((n))
counts_2d = np.zeros((n,n))
counts_3d = np.zeros((n,n,n))

for i in range(length):
    if True:
        print(i)
        file_text = train_1[i]["codes"]
        tree = javalang.parse.parse(file_text)
        tokens = token_list_2(tree)
        d = 0
        print_mode = 0
        path_1 = []
        bigrams = []
        bigram_list(tree,d,path_1,print_mode)
        s_counts_array_1 = compute_counts(tokens)
        s_counts_array_2d_1 = compute_counts_2d_s(bigrams)
        
        d = 0
        path_1 = []
        trigrams = []
        trigram_list(tree,d,path_1,print_mode)
        s_counts_array_3d_1 = compute_counts_3d_s(trigrams)

print('words list:\n',words,'\n')
print(s_counts_array_1)
print(s_counts_array_2d_1)

In [ ]:
# methods
n = len(words)
length = len(train_1)
counts = np.zeros((n))
counts_2d = np.zeros((n,n))

for i in range(length):
    if True:
        print(i)
        file_text = train_1[i]["codes"]
        tree = parse_m(file_text)
        tokens = token_list_2(tree)
        d = 0
        print_mode = 0
        path_1 = []
        bigrams = []
        bigram_list(tree,d,path_1,print_mode)
        s_counts_array_1 = compute_counts(tokens)
        s_counts_array_2d_1 = compute_counts_2d_s(bigrams)

print('words list:\n',words,'\n')
print(s_counts_array_1)
print(s_counts_array_2d_1)

In [ ]:
# files
n = len(words)
length = len(train_2)
counts = np.zeros((n))
counts_2d = np.zeros((n,n))
counts_3d = np.zeros((n,n,n))

for i in range(length):
    if True:
        print(i)
        file_text = train_2[i]["codes"]
        tree = javalang.parse.parse(file_text)
        tokens = token_list_2(tree)
        d = 0
        print_mode = 0
        path_1 = []
        bigrams = []
        bigram_list(tree,d,path_1,print_mode)
        s_counts_array_2 = compute_counts(tokens)
        s_counts_array_2d_2 = compute_counts_2d_s(bigrams)
        
        d = 0
        path_1 = []
        trigrams = []
        trigram_list(tree,d,path_1,print_mode)
        s_counts_array_3d_2 = compute_counts_3d_s(trigrams)


print('words list:\n',words,'\n')
print(s_counts_array_2)
print(s_counts_array_2d_2)

In [ ]:
# methods
n = len(words)
length = len(train_2)
counts = np.zeros((n))
counts_2d = np.zeros((n,n))

for i in range(length):
    if True:
        print(i)
        file_text = train_2[i]["codes"]
        tree = parse_m(file_text)
        tokens = token_list_2(tree)
        d = 0
        print_mode = 0
        path_1 = []
        bigrams = []
        bigram_list(tree,d,path_1,print_mode)
        s_counts_array_2 = compute_counts(tokens)
        s_counts_array_2d_2 = compute_counts_2d_s(bigrams)

print('words list:\n',words,'\n')
print(s_counts_array_2)
print(s_counts_array_2d_2)

In [ ]:
s_unigram_1 = compute_unigram_without_smoothing(s_counts_array_1)

s_unigram_2 = compute_unigram_without_smoothing(s_counts_array_2)

s_bigram_1 = compute_bigram_without_smoothing_s(s_counts_array_2d_1)

s_bigram_2 = compute_bigram_without_smoothing_s(s_counts_array_2d_2)

print('words list:\n',words,'\n')
s_trigram_1 = compute_trigram_without_smoothing_s(s_counts_array_3d_1)
print('trigram_1:\n',s_trigram_1[4])

print('words list:\n',words,'\n')
s_trigram_2 = compute_trigram_without_smoothing_s(s_counts_array_3d_2)
print('trigram_2:\n',s_trigram_2[4])

In [ ]:
# files
np.save('./data_2/s_unigram_1.npy',s_unigram_1)
np.save('./data_2/s_unigram_2.npy',s_unigram_2)

np.save('./data_2/s_bigram_1.npy',s_bigram_1)
np.save('./data_2/s_bigram_2.npy',s_bigram_2)

In [ ]:
# methods
np.save('./data_2/s_unigram_1_m.npy',s_unigram_1)
np.save('./data_2/s_unigram_2_m.npy',s_unigram_2)

np.save('./data_2/s_bigram_1_m.npy',s_bigram_1)
np.save('./data_2/s_bigram_2_m.npy',s_bigram_2)

In [ ]:
# files
s_unigram_1 = np.load('./data_2/s_unigram_1.npy')
s_unigram_2 = np.load('./data_2/s_unigram_2.npy')

s_bigram_1 = np.load('./data_2/s_bigram_1.npy')
s_bigram_2 = np.load('./data_2/s_bigram_2.npy')

In [ ]:
# methods
s_unigram_1 = np.load('./data_2/s_unigram_1_m.npy')
s_unigram_2 = np.load('./data_2/s_unigram_2_m.npy')

s_bigram_1 = np.load('./data_2/s_bigram_1_m.npy')
s_bigram_2 = np.load('./data_2/s_bigram_2_m.npy')

## Storing indices of non_zero bigrams (for calculating a dense bigram array):

In [ ]:
AST_type = 4

In [ ]:
ngram_type = 2
if (ngram_type == 1 or AST_type != 0):
    b_1 = bigram_1.copy()
    b_2 = bigram_2.copy()
elif (ngram_type == 2):
    b_1 = s_bigram_1.copy()
    b_2 = s_bigram_2.copy()

dense_bigram_1 = []
dense_bigram_2 = []

for i in range(len(words)):
    for j in range(len(words)):
        if (b_1[i,j]!=0 and b_2[i,j]!=0):
            dense_bigram_1.append([i,j,b_1[i,j]])
            dense_bigram_2.append([i,j,b_2[i,j]])

dense_bigram_1 = np.array(dense_bigram_1)
dense_bigram_2 = np.array(dense_bigram_2)

print(len(dense_bigram_1))
print(len(dense_bigram_2))

In [ ]:
np.save('./data_2/dense_bigram_1_AST4.npy',dense_bigram_1)

In [ ]:
# files
if (ngram_type == 1):
    np.save('./data_2/dense_bigram_1.npy',dense_bigram_1)
if (ngram_type == 2):
    np.save('./data_2/s_dense_bigram_1.npy',dense_bigram_1)

In [ ]:
# methods
if (ngram_type == 1):
    np.save('./data_2/dense_bigram_1_m.npy',dense_bigram_1)
if (ngram_type == 2):
    np.save('./data_2/s_dense_bigram_1_m.npy',dense_bigram_1)

In [ ]:
dense_bigram_1 = np.load('./data_2/dense_bigram_1_AST4.npy')

In [ ]:
# files
dense_bigram_1 = np.load('./data_2/dense_bigram_1.npy')

In [ ]:
# methods
dense_bigram_1 = np.load('./data_2/dense_bigram_1_m.npy')
s_dense_bigram_1 = np.load('./data_2/s_dense_bigram_1_m.npy')

# Storing dense trigrams:

In [ ]:
# trigrams
ngram_type = 1
if (ngram_type == 1 or AST_type != 0):
    t_1 = trigram_1.copy()
    t_2 = trigram_2.copy()
elif (ngram_type == 2):
    t_1 = s_trigram_1.copy()
    t_2 = s_trigram_2.copy()

dense_trigram_1 = []
dense_trigram_2 = []

for i in range(len(words)):
    for j in range(len(words)):
        for k in range(len(words)):
            if (t_1[i,j,k]!=0 and t_2[i,j,k]!=0):
                dense_trigram_1.append([i,j,k,t_1[i,j,k]])
                dense_trigram_2.append([i,j,k,t_2[i,j,k]])

dense_trigram_1 = np.array(dense_trigram_1)
dense_trigram_2 = np.array(dense_trigram_2)

print(len(dense_trigram_1))
print(len(dense_trigram_2))

In [ ]:
np.save('./data_2/dense_trigram_1_AST4.npy',dense_trigram_1)

In [ ]:
# files
if (ngram_type == 1):
    np.save('./data_2/dense_trigram_1.npy',dense_trigram_1)
if (ngram_type == 2):
    np.save('./data_2/s_dense_trigram_1.npy',dense_trigram_1)

In [ ]:
dense_trigram_1 = np.load('./data_2/dense_trigram_1_AST4.npy')

In [ ]:
# files
dense_trigram_1 = np.load('./data_2/dense_trigram_1.npy')

# Computing ngrams probability & count for each text file:

In [ ]:
data = train_1 + train_2
print(len(data))
print(data[100]["is_buggy"])
random.Random(0).shuffle(data)
print(data[4]["is_buggy"])
print(data[0]["codes"])

In [ ]:
with open("./data_2/d_3.txt","wb") as fp:
    pickle.dump(data, fp)

In [ ]:
with open("./data_2/d_3.txt","rb") as fp:
    data = pickle.load(fp)

In [ ]:
def compute_counts_file(tokens):
    counts = np.zeros((n))
    for i in range(len(tokens)):
        word_existed = 0
        for j in range(len(words)):
            if (tokens[i] == words[j]):
                counts[j]+=1
                word_existed = 1
                break
        if (word_existed == 0):
            counts[n-1]+=1
    return counts

def compute_counts_2d_file(tokens):
    counts_2d = np.zeros((n,n))
    for i in range(len(tokens)-1):
        i_1 = -1
        i_2 = -1
        for j in range(len(words)):
            if (tokens[i] == words[j]):
                i_1 = j
            if (tokens[i+1] == words[j]):
                i_2 = j
        if (i_1 == -1):
            i_1 = n-1
        if (i_2 == -1):
            i_2 = n-1
        counts_2d[i_1,i_2] += 1
    return counts_2d

def compute_counts_2d_file_s(b):
    counts_2d = np.zeros((n,n))
    for i in range(len(b)):
        i_1 = -1
        i_2 = -1
        for j in range(len(words)):
            if (b[i][0] == words[j]):
                i_1 = j
            if (b[i][1] == words[j]):
                i_2 = j
        if (i_1 == -1):
            i_1 = n-1
        if (i_2 == -1):
            i_2 = n-1
        counts_2d[i_1,i_2] += 1
    return counts_2d

def compute_counts_3d_file(tokens):
    counts_3d = np.zeros((n,n,n))
    for i in range(len(tokens)-2):
        i_1 = -1
        i_2 = -1
        i_3 = -1
        for j in range(len(words)):
            if (tokens[i] == words[j]):
                i_1 = j
            if (tokens[i+1] == words[j]):
                i_2 = j
            if (tokens[i+2] == words[j]):
                i_3 = j
        if (i_1 == -1):
            i_1 = n-1
        if (i_2 == -1):
            i_2 = n-1
        if (i_3 == -1):
            i_3 = n-1
        counts_3d[i_1,i_2,i_3] += 1
    return counts_3d

def compute_counts_3d_file_s(t):
    counts_3d = np.zeros((n,n,n))
    for i in range(len(t)):
        i_1 = -1
        i_2 = -1
        i_3 = -1
        for j in range(len(words)):
            if (t[i][0] == words[j]):
                i_1 = j
            if (t[i][1] == words[j]):
                i_2 = j
            if (t[i][2] == words[j]):
                i_3 = j
        if (i_1 == -1):
            i_1 = n-1
        if (i_2 == -1):
            i_2 = n-1
        if (i_3 == -1):
            i_3 = n-1
        counts_3d[i_1,i_2,i_3] += 1
    return counts_3d

## Computing ngrams probability & count for each text file (ngram_type = 1 or serial AST):

In [ ]:
print(AST_type)

In [ ]:
print(ngram_type)

In [ ]:
# files
n = len(words)
unigram_data = []
bigram_data = []
dense_bigram_data = []
dense_trigram_data = []

unigram_data_c = []
bigram_data_c = []
dense_bigram_data_c = []
dense_trigram_data_c = []

for i in range(len(data)):
    if True:
        print(i)
        counts = np.zeros((n))
        counts_2d = np.zeros((n,n))
        counts_3d = np.zeros((n,n,n))
        file_text = data[i]["codes"]
    
        if (AST_type == 0 and ngram_type == 1):
            p_file_text = preprocess(file_text)
            tokens = token_list(p_file_text)
        else:
            tree = javalang.parse.parse(file_text)
            i_2 = 0
            print_mode = 0
            path_1 = []
            tree_2 = create_tree(tree,i_2,path_1,print_mode)
            if (AST_type == 1):
                tokens = [n.name for n in LevelOrderIter(tree_2)]
            elif (AST_type == 2):
                tokens = [n.name for n in PreOrderIter(tree_2)]
            elif (AST_type == 3):
                tokens = [n.name for n in PostOrderIter(tree_2)]
    
        counts = compute_counts_file(tokens)
        counts_2d = compute_counts_2d_file(tokens)
        counts_3d = compute_counts_3d_file(tokens)
        unigram = compute_unigram_without_smoothing(counts)
        bigram = compute_bigram_without_smoothing(counts_2d,counts)
        trigram = compute_trigram_without_smoothing(counts_3d,counts_2d)
        
        a = []
        b = []
        c = []
        d = []
        
        a_c = []
        b_c = []
        c_c = []
        d_c = []
        
        for k in range(len(dense_bigram_1)):
            k_1 = int(dense_bigram_1[k,0])
            k_2 = int(dense_bigram_1[k,1])
            
            c.append(bigram[k_1,k_2])
            c_c.append(counts_2d[k_1,k_2])
            
        for k in range(len(dense_trigram_1)):
            k_1 = int(dense_trigram_1[k,0])
            k_2 = int(dense_trigram_1[k,1])
            k_3 = int(dense_trigram_1[k,2])
            
            d.append(trigram[k_1,k_2,k_3])
            d_c.append(counts_3d[k_1,k_2,k_3])
           
        a = unigram
        b = bigram.flatten()
        c = np.array(c)
        d = np.array(d)
        
        a_c = counts
        b_c = counts_2d.flatten()
        c_c = np.array(c_c)
        d_c = np.array(d_c)
        
        if (data[i]["is_buggy"]==True):
            a = np.append(a,1)
            b = np.append(b,1)
            c = np.append(c,1)
            d = np.append(d,1)
            
            a_c = np.append(a_c,1)
            b_c = np.append(b_c,1)
            c_c = np.append(c_c,1)
            d_c = np.append(d_c,1)
        
        else:
            a = np.append(a,0)
            b = np.append(b,0)
            c = np.append(c,0)
            d = np.append(d,0)
            
            a_c = np.append(a_c,0)
            b_c = np.append(b_c,0)
            c_c = np.append(c_c,0)
            d_c = np.append(d_c,0)

        unigram_data.append(a)
        bigram_data.append(b)
        dense_bigram_data.append(c)
        dense_trigram_data.append(d)
        
        unigram_data_c.append(a_c)
        bigram_data_c.append(b_c)
        dense_bigram_data_c.append(c_c)
        dense_trigram_data_c.append(d_c)
                
unigram_data = np.array(unigram_data)
dense_bigram_data = np.array(dense_bigram_data)
dense_trigram_data = np.array(dense_trigram_data)

unigram_data_c = np.array(unigram_data_c)
dense_bigram_data_c = np.array(dense_bigram_data_c)
dense_trigram_data_c = np.array(dense_trigram_data_c)

In [ ]:
# Time
# files
n = len(words)
unigram_data = []
bigram_data = []
dense_bigram_data = []
dense_trigram_data = []

unigram_data_c = []
bigram_data_c = []
dense_bigram_data_c = []
dense_trigram_data_c = []

bigram_computing_time = 0
trigram_computing_time = 0

for i in range(len(data)):
    if True:
        print(i)
        counts = np.zeros((n))
        counts_2d = np.zeros((n,n))
        counts_3d = np.zeros((n,n,n))
        file_text = data[i]["codes"]
    
        if (AST_type == 0 and ngram_type == 1):
            p_file_text = preprocess(file_text)
            tokens = token_list(p_file_text)
        else:
            tree = javalang.parse.parse(file_text)
            i_2 = 0
            print_mode = 0
            path_1 = []
            tree_2 = create_tree(tree,i_2,path_1,print_mode)
            if (AST_type == 1):
                tokens = [n.name for n in LevelOrderIter(tree_2)]
            elif (AST_type == 2):
                tokens = [n.name for n in PreOrderIter(tree_2)]
            elif (AST_type == 3):
                tokens = [n.name for n in PostOrderIter(tree_2)]
    
        counts = compute_counts_file(tokens)
        counts_2d = compute_counts_2d_file(tokens)
        counts_3d = compute_counts_3d_file(tokens)
        unigram = compute_unigram_without_smoothing(counts)
        
        b_1_t = time.time()
        bigram = compute_bigram_without_smoothing_d(counts_2d,counts)
        b_2_t = time.time()
        bigram_computing_time += (b_2_t -b_1_t)
        
        t_1_t = time.time()
        trigram = compute_trigram_without_smoothing_d(counts_3d,counts_2d)
        t_2_t = time.time()
        trigram_computing_time += (t_2_t -t_1_t)
        
        a = []
        b = []
        c = []
        d = []
        
        a_c = []
        b_c = []
        c_c = []
        d_c = []
        
        for k in range(len(dense_bigram_1)):
            k_1 = int(dense_bigram_1[k,0])
            k_2 = int(dense_bigram_1[k,1])
            
            c.append(bigram[k_1,k_2])
            c_c.append(counts_2d[k_1,k_2])
            
        for k in range(len(dense_trigram_1)):
            k_1 = int(dense_trigram_1[k,0])
            k_2 = int(dense_trigram_1[k,1])
            k_3 = int(dense_trigram_1[k,2])
            
            d.append(trigram[k_1,k_2,k_3])
            d_c.append(counts_3d[k_1,k_2,k_3])
           
        a = unigram
        b = bigram.flatten()
        c = np.array(c)
        d = np.array(d)
        
        a_c = counts
        b_c = counts_2d.flatten()
        c_c = np.array(c_c)
        d_c = np.array(d_c)
        
        if (data[i]["is_buggy"]==True):
            a = np.append(a,1)
            b = np.append(b,1)
            c = np.append(c,1)
            d = np.append(d,1)
            
            a_c = np.append(a_c,1)
            b_c = np.append(b_c,1)
            c_c = np.append(c_c,1)
            d_c = np.append(d_c,1)
        
        else:
            a = np.append(a,0)
            b = np.append(b,0)
            c = np.append(c,0)
            d = np.append(d,0)
            
            a_c = np.append(a_c,0)
            b_c = np.append(b_c,0)
            c_c = np.append(c_c,0)
            d_c = np.append(d_c,0)

        unigram_data.append(a)
        bigram_data.append(b)
        dense_bigram_data.append(c)
        dense_trigram_data.append(d)
        
        unigram_data_c.append(a_c)
        bigram_data_c.append(b_c)
        dense_bigram_data_c.append(c_c)
        dense_trigram_data_c.append(d_c)
                
unigram_data = np.array(unigram_data)
dense_bigram_data = np.array(dense_bigram_data)
dense_trigram_data = np.array(dense_trigram_data)

unigram_data_c = np.array(unigram_data_c)
dense_bigram_data_c = np.array(dense_bigram_data_c)
dense_trigram_data_c = np.array(dense_trigram_data_c)

print('bigram computing time:',round(bigram_computing_time,4))
print('trigram computing time:',round(trigram_computing_time,4))

In [ ]:
# methods
n = len(words)
unigram_data = []
bigram_data = []
dense_bigram_data = []

for i in range(len(data)):
    if True:
        print(i)
        counts = np.zeros((n))
        counts_2d = np.zeros((n,n))
        file_text = data[i]["codes"]
        p_file_text = preprocess(file_text)
        tokens = token_list(p_file_text)
        counts = compute_counts_file(tokens)
        counts_2d = compute_counts_2d_file(tokens)
        unigram = compute_unigram_without_smoothing(counts)
        bigram = compute_bigram_without_smoothing(counts_2d,counts)
        
        a = []
        b = []
        c = []
        for k in range(len(dense_bigram_1)):
            k_1 = int(dense_bigram_1[k,0])
            k_2 = int(dense_bigram_1[k,1])
            c.append(bigram[k_1,k_2])
           
        a = unigram
        b = bigram.flatten()
        c = np.array(c)
        
        if (data[i]["is_buggy"]==True):
            a = np.append(a,1)
            b = np.append(b,1)
            c = np.append(c,1)
        else:
            a = np.append(a,0)
            b = np.append(b,0)
            c = np.append(c,0)

        unigram_data.append(a)
        bigram_data.append(b)
        dense_bigram_data.append(c)
                
unigram_data = np.array(unigram_data)
dense_bigram_data = np.array(dense_bigram_data)

In [ ]:
# files (words)
np.save('./data_3/unigram_data.npy',unigram_data)
np.save('./data_3/dense_bigram_data.npy',dense_bigram_data)
np.save('./data_3/dense_trigram_data.npy',dense_trigram_data)

np.save('./data_3/unigram_data_c.npy',unigram_data_c)
np.save('./data_3/dense_bigram_data_c.npy',dense_bigram_data_c)
np.save('./data_3/dense_trigram_data_c.npy',dense_trigram_data_c)

In [ ]:
# files (AST1)
np.save('./data_2/unigram_data_AST1.npy',unigram_data)
np.save('./data_2/dense_bigram_data_AST1.npy',dense_bigram_data)
np.save('./data_2/dense_trigram_data_AST1.npy',dense_trigram_data)

np.save('./data_2/unigram_data_c_AST1.npy',unigram_data_c)
np.save('./data_2/dense_bigram_data_c_AST1.npy',dense_bigram_data_c)
np.save('./data_2/dense_trigram_data_c_AST1.npy',dense_trigram_data_c)

In [ ]:
# files (AST2)
np.save('./data_2/unigram_data_AST2.npy',unigram_data)
np.save('./data_2/dense_bigram_data_AST2.npy',dense_bigram_data)
np.save('./data_2/dense_trigram_data_AST2.npy',dense_trigram_data)

np.save('./data_2/unigram_data_c_AST2.npy',unigram_data_c)
np.save('./data_2/dense_bigram_data_c_AST2.npy',dense_bigram_data_c)
np.save('./data_2/dense_trigram_data_c_AST2.npy',dense_trigram_data_c)

In [ ]:
# files (AST3)
np.save('./data_2/unigram_data_AST3.npy',unigram_data)
np.save('./data_2/dense_bigram_data_AST3.npy',dense_bigram_data)
np.save('./data_2/dense_trigram_data_AST3.npy',dense_trigram_data)

np.save('./data_2/unigram_data_c_AST3.npy',unigram_data_c)
np.save('./data_2/dense_bigram_data_c_AST3.npy',dense_bigram_data_c)
np.save('./data_2/dense_trigram_data_c_AST3.npy',dense_trigram_data_c)

In [ ]:
# methods
np.save('./data_2/unigram_data_m.npy',unigram_data)
np.save('./data_2/dense_bigram_data_m.npy',dense_bigram_data)

In [ ]:
# files (words)
unigram_data = np.load('./data_2/unigram_data.npy')
dense_bigram_data = np.load('./data_2/dense_bigram_data.npy')
dense_trigram_data = np.load('./data_2/dense_trigram_data.npy')

unigram_data_c = np.load('./data_2/unigram_data_c.npy')
dense_bigram_data_c = np.load('./data_2/dense_bigram_data_c.npy')
dense_trigram_data_c = np.load('./data_2/dense_trigram_data_c.npy')

In [ ]:
# files (AST1)
unigram_data = np.load('./data_2/unigram_data_AST1.npy')
dense_bigram_data = np.load('./data_2/dense_bigram_data_AST1.npy')
dense_trigram_data = np.load('./data_2/dense_trigram_data_AST1.npy')

unigram_data_c = np.load('./data_2/unigram_data_c_AST1.npy')
dense_bigram_data_c = np.load('./data_2/dense_bigram_data_c_AST1.npy')
dense_trigram_data_c = np.load('./data_2/dense_trigram_data_c_AST1.npy')

In [ ]:
# files (AST2)
unigram_data = np.load('./data_2/unigram_data_AST2.npy')
dense_bigram_data = np.load('./data_2/dense_bigram_data_AST2.npy')
dense_trigram_data = np.load('./data_2/dense_trigram_data_AST2.npy')

unigram_data_c = np.load('./data_2/unigram_data_c_AST2.npy')
dense_bigram_data_c = np.load('./data_2/dense_bigram_data_c_AST2.npy')
dense_trigram_data_c = np.load('./data_2/dense_trigram_data_c_AST2.npy')

In [ ]:
# files (AST3)
unigram_data = np.load('./data_2/unigram_data_AST3.npy')
dense_bigram_data = np.load('./data_2/dense_bigram_data_AST3.npy')
dense_trigram_data = np.load('./data_2/dense_trigram_data_AST3.npy')

unigram_data_c = np.load('./data_2/unigram_data_c_AST3.npy')
dense_bigram_data_c = np.load('./data_2/dense_bigram_data_c_AST3.npy')
dense_trigram_data_c = np.load('./data_2/dense_trigram_data_c_AST3.npy')

In [ ]:
# methods
unigram_data = np.load('./data_2/unigram_data_m.npy')
dense_bigram_data = np.load('./data_2/dense_bigram_data_m.npy')

## Computing ngrams probability & count for each text file (path-based AST):

In [ ]:
print(AST_type)

In [ ]:
print(ngram_type)

In [ ]:
# files
n = len(words)
unigram_data = []
bigram_data = []
dense_bigram_data = []
dense_trigram_data = []

unigram_data_c = []
bigram_data_c = []
dense_bigram_data_c = []
dense_trigram_data_c = []

for i in range(len(data)):
    if True:
        print(i)
        counts = np.zeros((n))
        counts_2d = np.zeros((n,n))
        counts_3d = np.zeros((n,n,n))
        file_text = data[i]["codes"]
        tree = javalang.parse.parse(file_text)
        tokens = token_list_2(tree)
        d = 0
        print_mode = 0
        path_1 = []
        bigrams = []
        bigram_list(tree,d,path_1,print_mode)
        
        d = 0
        path_1 = []
        trigrams = []
        print_mode = 0
        trigram_list(tree,d,path_1,print_mode)
        
        counts = compute_counts_file(tokens)
        counts_2d = compute_counts_2d_file_s(bigrams)
        counts_3d = compute_counts_3d_file_s(trigrams)
        unigram = compute_unigram_without_smoothing(counts)
        bigram = compute_bigram_without_smoothing_s(counts_2d)
        trigram = compute_trigram_without_smoothing_s(counts_3d)
        
        a = []
        b = []
        c = []
        d = []
        
        a_c = []
        b_c = []
        c_c = []
        d_c = []
        
        for k in range(len(dense_bigram_1)):
            k_1 = int(dense_bigram_1[k,0])
            k_2 = int(dense_bigram_1[k,1])
            
            c.append(bigram[k_1,k_2])
            c_c.append(counts_2d[k_1,k_2])
            
        for k in range(len(dense_trigram_1)):
            k_1 = int(dense_trigram_1[k,0])
            k_2 = int(dense_trigram_1[k,1])
            k_3 = int(dense_trigram_1[k,2])
            
            d.append(trigram[k_1,k_2,k_3])
            d_c.append(counts_3d[k_1,k_2,k_3])
        
        a = unigram
        b = bigram.flatten()
        c = np.array(c)
        d = np.array(d)
        
        a_c = counts
        b_c = counts_2d.flatten()
        c_c = np.array(c_c)
        d_c = np.array(d_c)
        
        if (data[i]["is_buggy"]==True):
            a = np.append(a,1)
            b = np.append(b,1)
            c = np.append(c,1)
            d = np.append(d,1)
            
            a_c = np.append(a_c,1)
            b_c = np.append(b_c,1)
            c_c = np.append(c_c,1)
            d_c = np.append(d_c,1)
        
        else:
            a = np.append(a,0)
            b = np.append(b,0)
            c = np.append(c,0)
            d = np.append(d,0)
            
            a_c = np.append(a_c,0)
            b_c = np.append(b_c,0)
            c_c = np.append(c_c,0)
            d_c = np.append(d_c,0)


        unigram_data.append(a)
        bigram_data.append(b)
        dense_bigram_data.append(c)
        dense_trigram_data.append(d)
        
        unigram_data_c.append(a_c)
        bigram_data_c.append(b_c)
        dense_bigram_data_c.append(c_c)
        dense_trigram_data_c.append(d_c)
                
unigram_data = np.array(unigram_data)
dense_bigram_data = np.array(dense_bigram_data)
dense_trigram_data = np.array(dense_trigram_data)

unigram_data_c = np.array(unigram_data_c)
dense_bigram_data_c = np.array(dense_bigram_data_c)
dense_trigram_data_c = np.array(dense_trigram_data_c)

In [ ]:
# methods
n = len(words)
s_unigram_data = []
s_bigram_data = []
s_dense_bigram_data = []

for i in range(len(data)):
    if True:
        print(i)
        counts = np.zeros((n))
        counts_2d = np.zeros((n,n))
        file_text = data[i]["codes"]
        tree = parse_m(file_text)
        tokens = token_list_2(tree)
        d = 0
        print_mode = 0
        path_1 = []
        bigrams = []
        bigram_list(tree,d,path_1,print_mode)
        counts = compute_counts_file(tokens)
        counts_2d = compute_counts_2d_file_s(bigrams)
        unigram = compute_unigram_without_smoothing(counts)
        bigram = compute_bigram_without_smoothing_s(counts_2d)
        
        a = []
        b = []
        c = []
        for k in range(len(s_dense_bigram_1)):
            k_1 = int(s_dense_bigram_1[k,0])
            k_2 = int(s_dense_bigram_1[k,1])
            c.append(bigram[k_1,k_2])
           
        a = unigram
        b = bigram.flatten()
        c = np.array(c)
        
        if (data[i]["is_buggy"]==True):
            a = np.append(a,1)
            b = np.append(b,1)
            c = np.append(c,1)
        else:
            a = np.append(a,0)
            b = np.append(b,0)
            c = np.append(c,0)

        s_unigram_data.append(a)
        s_bigram_data.append(b)
        s_dense_bigram_data.append(c)
                
s_unigram_data = np.array(s_unigram_data)
s_dense_bigram_data = np.array(s_dense_bigram_data)

In [ ]:
# files (AST4)
np.save('./data_2/unigram_data_AST4.npy',unigram_data)
np.save('./data_2/dense_bigram_data_AST4.npy',dense_bigram_data)
np.save('./data_2/dense_trigram_data_AST4.npy',dense_trigram_data)

np.save('./data_2/unigram_data_c_AST4.npy',unigram_data_c)
np.save('./data_2/dense_bigram_data_c_AST4.npy',dense_bigram_data_c)
np.save('./data_2/dense_trigram_data_c_AST4.npy',dense_trigram_data_c)

In [ ]:
# methods
np.save('./data_2/s_unigram_data_m.npy',s_unigram_data)
np.save('./data_2/s_dense_bigram_data_m.npy',s_dense_bigram_data)

In [ ]:
# files (AST4)
unigram_data = np.load('./data_2/unigram_data_AST4.npy')
dense_bigram_data = np.load('./data_2/dense_bigram_data_AST4.npy')
dense_trigram_data = np.load('./data_2/dense_trigram_data_AST4.npy')

unigram_data_c = np.load('./data_2/unigram_data_c_AST4.npy')
dense_bigram_data_c = np.load('./data_2/dense_bigram_data_c_AST4.npy')
dense_trigram_data_c = np.load('./data_2/dense_trigram_data_c_AST4.npy')

In [ ]:
# methods
s_unigram_data = np.load('./data_2/s_unigram_data_m.npy')
s_dense_bigram_data = np.load('./data_2/s_dense_bigram_data_m.npy')

## Computing ngrams probability & count for each text file (second method path-based AST):

In [ ]:
print(AST_type)

In [ ]:
print(ngram_type)

In [ ]:
def compute_counts_file_s_bigrams(b):
    counts_bigram = np.zeros((n))
    counts_2d_bigram = np.zeros((n,n))
    for i in range(len(b)):
        i_1 = -1
        i_2 = -1
        for j in range(len(words)):
            if (b[i][0] == words[j]):
                i_1 = j
            if (b[i][1] == words[j]):
                i_2 = j
        if (i_1 == -1):
            i_1 = n-1
        if (i_2 == -1):
            i_2 = n-1
        
        counts_bigram[i_1] += 1
        counts_2d_bigram[i_1,i_2] += 1
        
    return counts_2d_bigram,counts_bigram

def compute_counts_file_s_trigrams(t):
    counts_2d_trigram = np.zeros((n,n))
    counts_3d_trigram = np.zeros((n,n,n))
    for i in range(len(t)):
        i_1 = -1
        i_2 = -1
        i_3 = -1
        for j in range(len(words)):
            if (t[i][0] == words[j]):
                i_1 = j
            if (t[i][1] == words[j]):
                i_2 = j
            if (t[i][2] == words[j]):
                i_3 = j
        if (i_1 == -1):
            i_1 = n-1
        if (i_2 == -1):
            i_2 = n-1
        if (i_3 == -1):
            i_3 = n-1
            
        counts_2d_trigram[i_1,i_2] += 1
        counts_3d_trigram[i_1,i_2,i_3] += 1
        
    return counts_3d_trigram,counts_2d_trigram

In [ ]:
# files
n = len(words)
unigram_data = []
bigram_data = []
dense_bigram_data = []
dense_trigram_data = []

unigram_data_c = []
bigram_data_c = []
dense_bigram_data_c = []
dense_trigram_data_c = []

for i in range(len(data)):
    if True:
        print(i)
        counts_unigram = np.zeros((n))
        counts_bigram = np.zeros((n))
        counts_2d_bigram = np.zeros((n,n))
        counts_2d_trigram = np.zeros((n,n))
        counts_3d_trigram = np.zeros((n,n,n))
        file_text = data[i]["codes"]
        tree = javalang.parse.parse(file_text)
        tokens = token_list_2(tree)
        d = 0
        print_mode = 0
        path_1 = []
        bigrams = []
        bigram_list(tree,d,path_1,print_mode)
        
        d = 0
        path_1 = []
        trigrams = []
        print_mode = 0
        trigram_list(tree,d,path_1,print_mode)
        
        counts_unigram = compute_counts_file(tokens)
        
        counts_2d_bigram,counts_bigram = compute_counts_file_s_bigrams(bigrams)
        
        counts_3d_trigram,counts_2d_trigram = compute_counts_file_s_trigrams(trigrams)
        
        unigram = compute_unigram_without_smoothing(counts_unigram)
        bigram = compute_bigram_without_smoothing(counts_2d_bigram,counts_bigram)
        trigram = compute_trigram_without_smoothing(counts_3d_trigram,counts_2d_trigram)
        
        a = []
        b = []
        c = []
        d = []
        
        a_c = []
        b_c = []
        c_c = []
        d_c = []
        
        for k in range(len(dense_bigram_1)):
            k_1 = int(dense_bigram_1[k,0])
            k_2 = int(dense_bigram_1[k,1])
            
            c.append(bigram[k_1,k_2])
            c_c.append(counts_2d_bigram[k_1,k_2])
            
        for k in range(len(dense_trigram_1)):
            k_1 = int(dense_trigram_1[k,0])
            k_2 = int(dense_trigram_1[k,1])
            k_3 = int(dense_trigram_1[k,2])
            
            d.append(trigram[k_1,k_2,k_3])
            d_c.append(counts_3d_trigram[k_1,k_2,k_3])
        
        a = unigram
        b = bigram.flatten()
        c = np.array(c)
        d = np.array(d)
        
        a_c = counts
        b_c = counts_2d.flatten()
        c_c = np.array(c_c)
        d_c = np.array(d_c)
        
        if (data[i]["is_buggy"]==True):
            a = np.append(a,1)
            b = np.append(b,1)
            c = np.append(c,1)
            d = np.append(d,1)
            
            a_c = np.append(a_c,1)
            b_c = np.append(b_c,1)
            c_c = np.append(c_c,1)
            d_c = np.append(d_c,1)
        
        else:
            a = np.append(a,0)
            b = np.append(b,0)
            c = np.append(c,0)
            d = np.append(d,0)
            
            a_c = np.append(a_c,0)
            b_c = np.append(b_c,0)
            c_c = np.append(c_c,0)
            d_c = np.append(d_c,0)


        unigram_data.append(a)
        bigram_data.append(b)
        dense_bigram_data.append(c)
        dense_trigram_data.append(d)
        
        unigram_data_c.append(a_c)
        bigram_data_c.append(b_c)
        dense_bigram_data_c.append(c_c)
        dense_trigram_data_c.append(d_c)
                
unigram_data = np.array(unigram_data)
dense_bigram_data = np.array(dense_bigram_data)
dense_trigram_data = np.array(dense_trigram_data)

unigram_data_c = np.array(unigram_data_c)
dense_bigram_data_c = np.array(dense_bigram_data_c)
dense_trigram_data_c = np.array(dense_trigram_data_c)

In [ ]:
# Time
# files
n = len(words)
unigram_data = []
bigram_data = []
dense_bigram_data = []
dense_trigram_data = []

unigram_data_c = []
bigram_data_c = []
dense_bigram_data_c = []
dense_trigram_data_c = []

bigram_computing_time = 0
trigram_computing_time = 0

for i in range(len(data)):
    if True:
        print(i)
        counts_unigram = np.zeros((n))
        counts_bigram = np.zeros((n))
        counts_2d_bigram = np.zeros((n,n))
        counts_2d_trigram = np.zeros((n,n))
        counts_3d_trigram = np.zeros((n,n,n))
        file_text = data[i]["codes"]
        tree = javalang.parse.parse(file_text)
        tokens = token_list_2(tree)
        d = 0
        print_mode = 0
        path_1 = []
        bigrams = []
        bigram_list(tree,d,path_1,print_mode)
        
        d = 0
        path_1 = []
        trigrams = []
        print_mode = 0
        trigram_list(tree,d,path_1,print_mode)
        
        counts_unigram = compute_counts_file(tokens)
        counts_2d_bigram,counts_bigram = compute_counts_file_s_bigrams(bigrams)
        counts_3d_trigram,counts_2d_trigram = compute_counts_file_s_trigrams(trigrams)
        
        unigram = compute_unigram_without_smoothing(counts_unigram)
        
        b_1_t = time.time()
        bigram = compute_bigram_without_smoothing_d(counts_2d_bigram,counts_bigram)
        b_2_t = time.time()
        bigram_computing_time += (b_2_t -b_1_t)
        
        t_1_t = time.time()
        trigram = compute_trigram_without_smoothing_d(counts_3d_trigram,counts_2d_trigram)
        t_2_t = time.time()
        trigram_computing_time += (t_2_t -t_1_t)
        
        a = []
        b = []
        c = []
        d = []
        
        a_c = []
        b_c = []
        c_c = []
        d_c = []
        
        for k in range(len(dense_bigram_1)):
            k_1 = int(dense_bigram_1[k,0])
            k_2 = int(dense_bigram_1[k,1])
            
            c.append(bigram[k_1,k_2])
            c_c.append(counts_2d_bigram[k_1,k_2])
            
        for k in range(len(dense_trigram_1)):
            k_1 = int(dense_trigram_1[k,0])
            k_2 = int(dense_trigram_1[k,1])
            k_3 = int(dense_trigram_1[k,2])
            
            d.append(trigram[k_1,k_2,k_3])
            d_c.append(counts_3d_trigram[k_1,k_2,k_3])
        
        a = unigram
        b = bigram.flatten()
        c = np.array(c)
        d = np.array(d)
        
        a_c = counts
        b_c = counts_2d.flatten()
        c_c = np.array(c_c)
        d_c = np.array(d_c)
        
        if (data[i]["is_buggy"]==True):
            a = np.append(a,1)
            b = np.append(b,1)
            c = np.append(c,1)
            d = np.append(d,1)
            
            a_c = np.append(a_c,1)
            b_c = np.append(b_c,1)
            c_c = np.append(c_c,1)
            d_c = np.append(d_c,1)
        
        else:
            a = np.append(a,0)
            b = np.append(b,0)
            c = np.append(c,0)
            d = np.append(d,0)
            
            a_c = np.append(a_c,0)
            b_c = np.append(b_c,0)
            c_c = np.append(c_c,0)
            d_c = np.append(d_c,0)


        unigram_data.append(a)
        bigram_data.append(b)
        dense_bigram_data.append(c)
        dense_trigram_data.append(d)
        
        unigram_data_c.append(a_c)
        bigram_data_c.append(b_c)
        dense_bigram_data_c.append(c_c)
        dense_trigram_data_c.append(d_c)
                
unigram_data = np.array(unigram_data)
dense_bigram_data = np.array(dense_bigram_data)
dense_trigram_data = np.array(dense_trigram_data)

unigram_data_c = np.array(unigram_data_c)
dense_bigram_data_c = np.array(dense_bigram_data_c)
dense_trigram_data_c = np.array(dense_trigram_data_c)

print('bigram computing time:',round(bigram_computing_time,4))
print('trigram computing time:',round(trigram_computing_time,4))

In [ ]:
# files (AST4)
np.save('./data_2/unigram_data_AST4.npy',unigram_data)
np.save('./data_2/dense_bigram_data_AST4.npy',dense_bigram_data)
np.save('./data_2/dense_trigram_data_AST4.npy',dense_trigram_data)

np.save('./data_2/unigram_data_c_AST4.npy',unigram_data_c)
np.save('./data_2/dense_bigram_data_c_AST4.npy',dense_bigram_data_c)
np.save('./data_2/dense_trigram_data_c_AST4.npy',dense_trigram_data_c)

In [ ]:
# files (AST4)
unigram_data = np.load('./data_2/unigram_data_AST4.npy')
dense_bigram_data = np.load('./data_2/dense_bigram_data_AST4.npy')
dense_trigram_data = np.load('./data_2/dense_trigram_data_AST4.npy')

unigram_data_c = np.load('./data_2/unigram_data_c_AST4.npy')
dense_bigram_data_c = np.load('./data_2/dense_bigram_data_c_AST4.npy')
dense_trigram_data_c = np.load('./data_2/dense_trigram_data_c_AST4.npy')

## Extracting CK metrics for each text file:

In [ ]:
# files
ck_metrics_data = []
y = 0
n = 0

for i in range(len(data)):
    if True:
        print(i)
        a = data[i]["ck_metrics"]
        a = [float(j) for j in a]
        
        if (data[i]["is_buggy"]==True):
            a.append(1)
            y += 1
        else:
            a.append(0)
            n += 1
            
        ck_metrics_data.append(a)
        
ck_metrics_data = np.array(ck_metrics_data)

print(y)
print(n)

In [ ]:
print(ck_metrics_data.shape)

In [ ]:
np.save('./data_2/ck_metrics_data.npy',ck_metrics_data)

In [ ]:
ck_metrics_data = np.load('./data_2/ck_metrics_data.npy')

# Defining a method to calculate metrics:

In [ ]:
def compute_metrics(Y_test,predicted_result):
    c_matrix = confusion_matrix(Y_test,predicted_result)
    print('confusion matrix:')
    print(c_matrix)
    tn = c_matrix[0,0]
    fp = c_matrix[0,1]
    fn = c_matrix[1,0]
    tp = c_matrix[1,1]
    
    if ((tp + fp) == 0):
        precision = 'not_defined'
    else:
        precision = tp / (tp + fp)
    if ((tp + fn) == 0):
        recall = 'not_defined'
    else:
        recall = tp / (tp + fn)
    
    accuracy = (tp + tn)/(tn + fp + fn + tp)
    
    if (precision == 'not_defined' or recall == 'not_defined' or (precision + recall) == 0):
        f1_score = 'not_defined'
    else:
        f1_score = (2 * precision * recall)/(precision + recall)
        
        
    if ((fp + tn) == 0):
        fp_rate = 'not_defined'
    else:
        fp_rate = fp / (fp + tn)

    print('precision:',precision)
    print('recall:',recall)
    print('accuracy:',accuracy)
    print('f1 score:',f1_score)
    print('FP rate:',fp_rate)
    return (precision,recall,accuracy,f1_score,fp_rate)

# Plotting the training accuracy and loss:

In [ ]:
def plot_variables(history):
    training_accuracy = history.history['acc']
    validation_accuracy = history.history['val_acc']
    training_loss = history.history['loss']
    validation_loss = history.history['val_loss']
    epochs = range(1,(epochs_num+1))

    plt.rcParams["figure.figsize"] = (10,6)
    plt.plot(epochs,training_accuracy,color='darkblue',label='training accuracy')
    plt.plot(epochs,validation_accuracy,color='red',label='validation accuracy')
    plt.title('Diagram of training accuracy and epochs',fontsize=16)
    plt.xlabel('epochs',fontsize=14)
    plt.ylabel('training accuracy',fontsize=14)
    plt.legend(fontsize=12)
    plt.show()

    plt.plot(epochs,training_loss,color='darkblue',label='training loss')
    plt.plot(epochs,validation_loss,color='red',label='validation loss')
    plt.title('Diagram of training loss and epochs',fontsize=16)
    plt.xlabel('epochs',fontsize=14)
    plt.ylabel('training loss',fontsize=14)
    plt.legend(fontsize=12)
    plt.show()

# Concatenating two datasets:

In [ ]:
def c_1(data_1,data_2):
    length_1 = data_1.shape[1]-1
    feature_1 = data_1[:,0:length_1]
    feature = np.concatenate((feature_1,data_2),axis=1)
    return feature

# Convolutional NN

In [ ]:
epochs_num = 8
def convolutional(data,mode):
    n_data = data.shape[0]
    length = data.shape[1] - 1
    c_result = data[:,length]
    feature = data[:,0:length]
    print(feature.shape)
    feature = feature.reshape(feature.shape[0],feature.shape[1],1)
    print(feature.shape)

    X_train, X_test, Y_train, Y_test = train_test_split(feature, c_result, test_size=0.1)

    if (mode == 1):
        model = tf.keras.Sequential()
        model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(length,1)))
        model.add(Dense(16, activation='relu'))
        model.add(MaxPooling1D())
        
        model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
        model.add(Dense(16, activation='relu'))
        model.add(MaxPooling1D())
        
        model.add(Flatten())
        model.add(Dense(2, activation='softmax'))

    print(model.summary())  
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    history = model.fit(X_train, Y_train, batch_size=512, epochs=epochs_num, validation_data=(X_test,Y_test))

    print('\nEvaluation:')
    model.evaluate(X_test,Y_test)
    
    predicted_r = model.predict(X_test)
    predicted_result = np.argmax(predicted_r,axis=1)
    Y_test = Y_test.astype(int)
    computed_metrics = compute_metrics(Y_test,predicted_result)

    return history

In [ ]:
history = convolutional(unigram_data,1)

In [ ]:
plot_variables(history)

# Dividing the data into fixed sets of test and train:

In [ ]:
def split_data_2(data):
    l_1 =data.shape[0]
    l_2 = int(np.ceil(l_1 / 10))
    random.seed(10)
    index_list = random.sample(range(l_1),l_2)
    test_data = []
    train_data = []
    for i in range(l_1):
        if i in index_list:
            test_data.append(data[i])
        else:
            train_data.append(data[i])
    test_data = np.array(test_data)
    train_data = np.array(train_data)
    return train_data, test_data

In [ ]:
train_1, test_1 = split_data_2(unigram_data)

# Plot ROC

In [ ]:
def plot_roc(fp_r,tp_r):
    plt.rcParams["figure.figsize"] = (3,3)
    plt.plot(fp_r,tp_r)
    plt.xlabel('FP rate')
    plt.ylabel('TP rate')
    plt.show()

# K-Fold splittig the data:

In [ ]:
def kf_data_split(data):
    kf_data = []
    kf_1 = KFold(n_splits = 10)
    for train_i, test_i in kf_1.split(data):
        train = data[train_i]
        test = data[test_i]
        kf_data.append([train,test])
    return(kf_data)

# Logistic regression with specified threshold and K-Fold cross-validation:

In [ ]:
def logistic_regression_3(train_1,test_1):
    max_iter = 1000000
    
#     threshold = 0.1623
    
    length = train_1.shape[1] - 1
    X_train = train_1[:,0:length]
    Y_train = train_1[:,length]
    X_test = test_1[:,0:length]
    Y_test = test_1[:,length]
    
    logistic_regression = LogisticRegression()
    model = logistic_regression.fit(X_train, Y_train)
    
    predicted_r = model.predict_proba(X_test)[:,1]
    
    fp_r, tp_r, t_array = roc_curve(Y_test,predicted_r)
        
    plot_roc(fp_r,tp_r)
    
    auc_1 = roc_auc_score(Y_test,predicted_r)
    print('AUC:',auc_1)
    
    print('threshold:',threshold)
    length_2 = int(predicted_r.shape[0])
    
    predicted_result = np.zeros(length_2)
    for i in range(length_2):
        if (predicted_r[i] >= threshold):
            predicted_result[i] = 1
        else:
            predicted_result[i] = 0
    
    Y_test = Y_test.astype(int)
    computed_metrics = compute_metrics(Y_test,predicted_result)
    return computed_metrics,auc_1

# Random forest with specified threshold and K-Fold cross-validation:

In [ ]:
def RF_classifier_3(train_1,test_1):
#     threshold = 0.1623
    length = train_1.shape[1] - 1
    X_train = train_1[:,0:length]
    Y_train = train_1[:,length]
    X_test = test_1[:,0:length]
    Y_test = test_1[:,length]
    
    clf = RandomForestClassifier(max_depth=m_depth,random_state=0)
    clf.fit(X_train, Y_train)

    predicted_r = clf.predict_proba(X_test)[:,1]
    
    fp_r, tp_r, t_array = roc_curve(Y_test,predicted_r)
        
    plot_roc(fp_r,tp_r)
    
    auc_1 = roc_auc_score(Y_test,predicted_r)
    print('AUC:',auc_1)
    
    print('threshold:',threshold)
    length_2 = int(predicted_r.shape[0])
    
    predicted_result = np.zeros(length_2)
    for i in range(length_2):
        if (predicted_r[i] >= threshold):
            predicted_result[i] = 1
        else:
            predicted_result[i] = 0
    
    Y_test = Y_test.astype(int)
    computed_metrics = compute_metrics(Y_test,predicted_result)
    return computed_metrics,auc_1

# Sequential NN with specified threshold and K-Fold cross-validation:

In [ ]:
epochs_num = 100
def sequential_3(train_1,test_1,mode):
#     threshold = 0.2052
    
    length = train_1.shape[1] - 1
    Y_train = train_1[:,length]
    Y_test = test_1[:,length]
    
    X_train = train_1[:,0:length]
    X_test = test_1[:,0:length]
    
    if (mode == 1):
        batch_size_1 = 8
        model = keras.Sequential([keras.layers.Dense(64, input_shape=(length,), activation='relu'), 
                                  keras.layers.Dropout(0.3), 
                                  keras.layers.Dense(32, activation='relu'), 
                                  keras.layers.Dropout(0.2), 
                                  keras.layers.Dense(8, activation='relu'),
                                  (keras.layers.Dense(1, activation='sigmoid'))])
    if (mode == 2):
        batch_size_1 = 8
        model = keras.Sequential([keras.layers.Dense(128, input_shape=(length,), activation='relu'), 
                                  keras.layers.Dropout(0.3), 
                                  keras.layers.Dense(32, activation='relu'), 
                                  keras.layers.Dropout(0.2), 
                                  keras.layers.Dense(16, activation='relu'),
                                  (keras.layers.Dense(1, activation='sigmoid'))])
    if (mode == 3):
        batch_size_1 = 8
        model = keras.Sequential([keras.layers.Dense(32, input_shape=(length,), activation='relu'), 
                                  keras.layers.Dropout(0.3), 
                                  keras.layers.Dense(8, activation='relu'), 
                                  (keras.layers.Dense(1, activation='sigmoid'))])
    if (mode == 4):
        batch_size_1 = 8
        model = keras.Sequential([keras.layers.Dense(128, input_shape=(length,), activation='relu'), 
                                  keras.layers.Dropout(0.3), 
                                  keras.layers.Dense(32, activation='relu'), 
                                  keras.layers.Dropout(0.2), 
                                  keras.layers.Dense(16, activation='relu'),
                                  (keras.layers.Dense(1, activation='sigmoid'))])
    if (mode == 5):
        batch_size_1 = 8
        model = keras.Sequential([keras.layers.Dense(256, input_shape=(length,), activation='relu'), 
                                  keras.layers.Dropout(0.3), 
                                  keras.layers.Dense(64, activation='relu'), 
                                  keras.layers.Dropout(0.2), 
                                  keras.layers.Dense(16, activation='relu'),
                                  (keras.layers.Dense(1, activation='sigmoid'))])
    if (mode == 6):
        batch_size_1 = 8
        model = keras.Sequential([keras.layers.Dense(16, input_shape=(length,), activation='relu'), 
                                  keras.layers.Dropout(0.3), 
                                  keras.layers.Dense(4, activation='relu'),
                                  (keras.layers.Dense(1, activation='sigmoid'))])

    model.compile(optimizer='adam', loss=keras.losses.BinaryCrossentropy(from_logits=False), metrics=['accuracy'])

    history = model.fit(X_train, Y_train, batch_size=batch_size_1, epochs=epochs_num, validation_data=(X_test,Y_test))

    print('\nEvaluation:')
    model.evaluate(X_test,Y_test)
    
    predicted_r = model.predict(X_test)
    
    fp_r, tp_r, t_array = roc_curve(Y_test,predicted_r)
    
    plot_roc(fp_r,tp_r)
    
    auc_1 = roc_auc_score(Y_test,predicted_r)
    print('AUC:',auc_1)
    
    print('threshold:',threshold)
    
    length_2 = int(predicted_r.shape[0])
    
    predicted_result = np.zeros(length_2)
    for i in range(length_2):
        if (predicted_r[i][0] >= threshold):
            predicted_result[i] = 1
        else:
            predicted_result[i] = 0
    
    Y_test = Y_test.astype(int)
    computed_metrics = compute_metrics(Y_test,predicted_result)
    
    return computed_metrics,auc_1

# K-nearest neighbors classifier:

In [ ]:
def KN_classifier_3(train_1,test_1):
#     threshold = 0.1623
    length = train_1.shape[1] - 1
    X_train = train_1[:,0:length]
    Y_train = train_1[:,length]
    X_test = test_1[:,0:length]
    Y_test = test_1[:,length]
    
    k_value = math.floor(math.sqrt(train_1.shape[0]))
    if (k_value % 2 == 0):
        print('e')
        k_value += 1
    else:
        print('o')
    print(k_value)
    
    model = KNeighborsClassifier(n_neighbors = k_value)
    model.fit(X_train, Y_train)

    predicted_r = model.predict_proba(X_test)[:,1]
    
    fp_r, tp_r, t_array = roc_curve(Y_test,predicted_r)
        
    plot_roc(fp_r,tp_r)
    
    auc_1 = roc_auc_score(Y_test,predicted_r)
    print('AUC:',auc_1)
    
    print('threshold:',threshold)
    length_2 = int(predicted_r.shape[0])
    
    predicted_result = np.zeros(length_2)
    for i in range(length_2):
        if (predicted_r[i] >= threshold):
            predicted_result[i] = 1
        else:
            predicted_result[i] = 0
    
    Y_test = Y_test.astype(int)
    computed_metrics = compute_metrics(Y_test,predicted_result)
    return computed_metrics,auc_1

# Computing average of metrics using K-Fold cross-validation (probabilistic classification):

In [ ]:
# Logistic regression => m:1
# Sequential NN => m:3,4
def kf_average(d_1,m):
    trials_n = 1
    precision_array = []
    recall_array = []
    accuracy_array = []
    f1_score_array = []
    fp_rate_array = []
    auc_array = []
    data = kf_data_split(d_1)
    for t in range(trials_n):
        print('\nTrial number:',t+1)
        for i in range(len(data)):
            print('\nIteration number:',(i+1))
            train_data = data[i][0]
            test_data = data[i][1]

            if (m == 1):
                result,auc_result = logistic_regression_3(train_data,test_data)
            if (m == 2):
                result,auc_result = RF_classifier_3(train_data,test_data)
            elif (m == 3):
                result,auc_result = sequential_3(train_data,test_data,1)
            elif (m == 4):
                result,auc_result = sequential_3(train_data,test_data,2)
            elif (m == 5):
                result,auc_result = sequential_3(train_data,test_data,3)
            elif (m == 6):
                result,auc_result = sequential_3(train_data,test_data,4)
            elif (m == 7):
                result,auc_result = sequential_3(train_data,test_data,5)
            elif (m == 8):
                result,auc_result = sequential_3(train_data,test_data,6)
            elif (m == 9):
                result,auc_result = KN_classifier_3(train_data,test_data)
                
            precision,recall,accuracy,f1_score,fp_rate = result

            if (precision != 'not_defined'):
                precision_array.append(precision)
            if (recall != 'not_defined'):
                recall_array.append(recall)
            if (accuracy != 'not_defined'):
                accuracy_array.append(accuracy)
            if (f1_score != 'not_defined'):
                f1_score_array.append(f1_score)
            if (fp_rate != 'not_defined'):
                fp_rate_array.append(fp_rate)
            if (auc_result != 'not_defined'):
                auc_array.append(auc_result)
    
    precision_array = np.array(precision_array)
    recall_array = np.array(recall_array)
    accuracy_array = np.array(accuracy_array)
    f1_score_array = np.array(f1_score_array)
    fp_rate_array = np.array(fp_rate_array)
    auc_array = np.array(auc_array)
    
    print('\nprecision array:',precision_array,'\naverage precision:',precision_array.mean(),'\n')
    print('\nrecall array:',recall_array,'\naverage recall:',recall_array.mean(),'\n')
    print('\naccuracy array:',accuracy_array,'\naverage accuracy:',accuracy_array.mean(),'\n')
    print('\nf1_score array:',f1_score_array,'\naverage f1_score:',f1_score_array.mean(),'\n')
    print('\nfp_rate array:',fp_rate_array,'\naverage fp_rate:',fp_rate_array.mean(),'\n')
    print('\nauc array:',auc_array,'\naverage auc:',auc_array.mean(),'\n')
    
    print('precision:',round(precision_array.mean(),4))
    print('recall:',round(recall_array.mean(),4))
    print('accuracy:',round(accuracy_array.mean(),4))
    print('f1 score:',round(f1_score_array.mean(),4))
    print('fp rate:',round(fp_rate_array.mean(),4))
    print('auc:',round(auc_array.mean(),4))

In [ ]:
# start
kf_average(b_data,1)

# Feature selection:

In [ ]:
def select_features_1(data,k_value,score_f):
    length = data.shape[1] - 1
    X = data[:,0:length]
    y = data[:,length].reshape(-1,1)
    if (score_f == 1):
        s = SelectKBest(score_func=f_classif, k=k_value)
    elif (score_f == 2):
        s = SelectKBest(score_func=chi2, k=k_value)
    elif (score_f == 3):
        s = SelectKBest(score_func=mutual_info_classif, k=k_value)
    elif (score_f == 4):
        s = SelectFpr(score_func=chi2, alpha=0.01)
    elif (score_f == 5):
        s = SelectFdr(score_func=chi2, alpha=0.01)
        
    new_X = s.fit_transform(X,y)
    columns = s.get_support(indices=True)
    scores = s.scores_[s.get_support()]
    new_data = np.hstack((new_X,y))
    
    zipped_f = zip(scores,columns)
    zipped_f = sorted(zipped_f,reverse=True)
    sorted_columns = [column for (score,column) in zipped_f]
    
    return new_data,sorted_columns

In [ ]:
def select_features_2(data,k_value,score_f):
    length = data.shape[1] - 1
    X = data[:,0:length]
    y = data[:,length].reshape(-1,1)
    
    if (score_f <= 5):
        if (score_f == 1):
            s = SelectKBest(score_func=f_classif, k=k_value)
        elif (score_f == 2):
            s = SelectKBest(score_func=chi2, k=k_value)
        elif (score_f == 3):
            s = SelectKBest(score_func=mutual_info_classif, k=k_value)
        elif (score_f == 4):
            s = SelectFpr(score_func=chi2, alpha=0.01)
        elif (score_f == 5):
            s = SelectFdr(score_func=chi2, alpha=0.01)
            
        new_X = s.fit_transform(X,y)
        columns = s.get_support(indices=True)
        scores = s.scores_[s.get_support()]
        new_data = np.hstack((new_X,y))

        zipped_f = zip(scores,columns)
        zipped_f = sorted(zipped_f,reverse=True)
        sorted_columns = [column for (score,column) in zipped_f]
    else:
        if (score_f == 6):
            estimator = SVR(kernel = "linear")
        elif (score_f == 7):
            estimator = LogisticRegression()
        elif (score_f == 8):
            estimator = RandomForestClassifier(max_depth=m_depth,random_state=0)
         
        print('step_value:',step_value)
        s = RFE(estimator,n_features_to_select=k_value,step=step_value)

        new_X = s.fit_transform(X,y)
        columns = s.get_support(indices=True)
        new_data = np.hstack((new_X,y))
        sorted_columns = [] 
    
    return new_data,sorted_columns
    

In [ ]:
# Mode=>   1:unigram,   2:dense_bigram,   3:s_unigram,   4: s_dense_bigram
def features_names(c,mode):
    f_names = []
    if (mode == 1):
        for i in range(len(c)):
            f_names.append(words[int(c[i])])
    
    elif (mode == 2):
        for i in range(len(c)):
            bigram_name = (words[int(dense_bigram_1[int(c[i])][0])],words[int(dense_bigram_1[int(c[i])][1])])
            f_names.append(bigram_name)
    
    elif (mode == 3):
        for i in range(len(c)):
            f_names.append(s_words[int(c[i])])
    
    elif (mode == 4):
        for i in range(len(c)):
            bigram_name = (s_words[int(s_dense_bigram_1[int(c[i])][0])],s_words[int(s_dense_bigram_1[int(c[i])][1])])
            f_names.append(bigram_name)
    
    return(f_names)

# Converting to binary array:

In [ ]:
def convert_to_binary(array_1):
    array_2 = np.where(array_1>0,1,0)
    return(array_2)

# Plot ROC-AUC and number of features:

In [ ]:
def compute_metrics_2(Y_test,predicted_result):
    c_matrix = confusion_matrix(Y_test,predicted_result)
    tn = c_matrix[0,0]
    fp = c_matrix[0,1]
    fn = c_matrix[1,0]
    tp = c_matrix[1,1]
    
    if ((tp + fp) == 0):
        precision = 'not_defined'
    else:
        precision = tp / (tp + fp)
    if ((tp + fn) == 0):
        recall = 'not_defined'
    else:
        recall = tp / (tp + fn)
    
    accuracy = (tp + tn)/(tn + fp + fn + tp)
    
    if (precision == 'not_defined' or recall == 'not_defined' or (precision + recall) == 0):
        f1_score = 'not_defined'
    else:
        f1_score = (2 * precision * recall)/(precision + recall)
        
        
    if ((fp + tn) == 0):
        fp_rate = 'not_defined'
    else:
        fp_rate = fp / (fp + tn)

    return (precision,recall,accuracy,f1_score,fp_rate)

In [ ]:
def logistic_regression_3_2(train_1,test_1):
    max_iter = 1000000

#     threshold = 0.1623
    
    length = train_1.shape[1] - 1
    X_train = train_1[:,0:length]
    Y_train = train_1[:,length]
    X_test = test_1[:,0:length]
    Y_test = test_1[:,length]
    
    logistic_regression = LogisticRegression()
    model = logistic_regression.fit(X_train, Y_train)
    
    predicted_r = model.predict_proba(X_test)[:,1]
    
    fp_r, tp_r, t_array = roc_curve(Y_test,predicted_r)
    
    auc_1 = roc_auc_score(Y_test,predicted_r)

    length_2 = int(predicted_r.shape[0])
    
    predicted_result = np.zeros(length_2)
    for i in range(length_2):
        if (predicted_r[i] >= threshold):
            predicted_result[i] = 1
        else:
            predicted_result[i] = 0
    
    Y_test = Y_test.astype(int)
    computed_metrics = compute_metrics_2(Y_test,predicted_result)
    return computed_metrics,auc_1

In [ ]:
def RF_classifier_3_2(train_1,test_1):
#     threshold = 0.1623
    length = train_1.shape[1] - 1
    X_train = train_1[:,0:length]
    Y_train = train_1[:,length]
    X_test = test_1[:,0:length]
    Y_test = test_1[:,length]
    
    clf = RandomForestClassifier(max_depth=m_depth,random_state=0)
    clf.fit(X_train, Y_train)

    predicted_r = clf.predict_proba(X_test)[:,1]
    
    fp_r, tp_r, t_array = roc_curve(Y_test,predicted_r)
        
    auc_1 = roc_auc_score(Y_test,predicted_r)

    length_2 = int(predicted_r.shape[0])
    
    predicted_result = np.zeros(length_2)
    for i in range(length_2):
        if (predicted_r[i] >= threshold):
            predicted_result[i] = 1
        else:
            predicted_result[i] = 0
    
    Y_test = Y_test.astype(int)
    computed_metrics = compute_metrics_2(Y_test,predicted_result)
    return computed_metrics,auc_1

In [ ]:
def KN_classifier_3_2(train_1,test_1):
#     threshold = 0.1623
    length = train_1.shape[1] - 1
    X_train = train_1[:,0:length]
    Y_train = train_1[:,length]
    X_test = test_1[:,0:length]
    Y_test = test_1[:,length]
    
    k_value = math.floor(math.sqrt(train_1.shape[0]))
    if (k_value % 2 == 0):
        k_value += 1
    else:
        start_val = 0
    
    model = KNeighborsClassifier(n_neighbors = k_value)
    model.fit(X_train, Y_train)

    predicted_r = model.predict_proba(X_test)[:,1]
    
    fp_r, tp_r, t_array = roc_curve(Y_test,predicted_r)
    
    auc_1 = roc_auc_score(Y_test,predicted_r)

    length_2 = int(predicted_r.shape[0])
    
    predicted_result = np.zeros(length_2)
    for i in range(length_2):
        if (predicted_r[i] >= threshold):
            predicted_result[i] = 1
        else:
            predicted_result[i] = 0
    
    Y_test = Y_test.astype(int)
    computed_metrics = compute_metrics_2(Y_test,predicted_result)
    return computed_metrics,auc_1

In [ ]:
def G_NB_3_2(train_1,test_1):
#     threshold = 0.1623
    length = train_1.shape[1] - 1
    X_train = train_1[:,0:length]
    Y_train = train_1[:,length]
    X_test = test_1[:,0:length]
    Y_test = test_1[:,length]
    
    clf = GaussianNB()
    clf.fit(X_train, Y_train)

    predicted_r = clf.predict_proba(X_test)[:,1]
    
    fp_r, tp_r, t_array = roc_curve(Y_test,predicted_r)
    
    auc_1 = roc_auc_score(Y_test,predicted_r)

    length_2 = int(predicted_r.shape[0])
    
    predicted_result = np.zeros(length_2)
    for i in range(length_2):
        if (predicted_r[i] >= threshold):
            predicted_result[i] = 1
        else:
            predicted_result[i] = 0
    
    Y_test = Y_test.astype(int)
    computed_metrics = compute_metrics_2(Y_test,predicted_result)
    return computed_metrics,auc_1

In [ ]:
# Logistic regression => m:1
# Sequential NN => m:3,4
def kf_average_2(d_1,m):
    trials_n = 1
    precision_array = []
    recall_array = []
    accuracy_array = []
    f1_score_array = []
    fp_rate_array = []
    auc_array = []
    data = kf_data_split(d_1)
    for t in range(trials_n):
        print('\nTrial number:',t+1)
        for i in range(len(data)):
            print('\nIteration number:',(i+1))
            train_data = data[i][0]
            test_data = data[i][1]

            if (m == 1):
                result,auc_result = logistic_regression_3_2(train_data,test_data)
            if (m == 2):
                result,auc_result = RF_classifier_3_2(train_data,test_data)
            elif (m == 3):
                result,auc_result = sequential_3(train_data,test_data,1)
            elif (m == 4):
                result,auc_result = sequential_3(train_data,test_data,2)
            elif (m == 5):
                result,auc_result = sequential_3(train_data,test_data,3)
            elif (m == 6):
                result,auc_result = sequential_3(train_data,test_data,4)
            elif (m == 7):
                result,auc_result = sequential_3(train_data,test_data,5)
            elif (m == 8):
                result,auc_result = sequential_3(train_data,test_data,6)
            elif (m == 9):
                result,auc_result = KN_classifier_3_2(train_data,test_data)
            elif (m == 10):
                result,auc_result = G_NB_3_2(train_data,test_data)
                
            precision,recall,accuracy,f1_score,fp_rate = result

            if (precision != 'not_defined'):
                precision_array.append(precision)
            if (recall != 'not_defined'):
                recall_array.append(recall)
            if (accuracy != 'not_defined'):
                accuracy_array.append(accuracy)
            if (f1_score != 'not_defined'):
                f1_score_array.append(f1_score)
            if (fp_rate != 'not_defined'):
                fp_rate_array.append(fp_rate)
            if (auc_result != 'not_defined'):
                auc_array.append(auc_result)
    
    precision_array = np.array(precision_array)
    recall_array = np.array(recall_array)
    accuracy_array = np.array(accuracy_array)
    f1_score_array = np.array(f1_score_array)
    fp_rate_array = np.array(fp_rate_array)
    auc_array = np.array(auc_array)
    
    mean_auc_value = round(auc_array.mean(),4)
    return(mean_auc_value)

In [ ]:
def compute_values(data_1,data_2,data_3,data_4,data_5,data_6,data_7,data_8,score_f_type,c_method):
    features_n = []
    
    auc_array_p_AST1 = []
    auc_array_c_AST1 = []
    auc_array_b_AST1 = []
    
    auc_array_p_AST2 = []
    auc_array_c_AST2 = []
    auc_array_b_AST2 = []
    
    auc_array_p_AST3 = []
    auc_array_c_AST3 = []
    auc_array_b_AST3 = []
    
    auc_array_p_AST4 = []
    auc_array_c_AST4 = []
    auc_array_b_AST4 = []
    for i in range(25,301,25):
        features_n.append(i)
        
        b_data = select_features_1(data_1,i,score_f_type)[0]
        auc_value_p = kf_average_2(b_data,c_method)
        auc_array_p_AST1.append(auc_value_p)
        
        b_data = select_features_1(data_2,i,score_f_type)[0]
        auc_value_c = kf_average_2(b_data,c_method)
        auc_array_c_AST1.append(auc_value_c)
        
        significant_b_data = select_features_1(data_2,i,score_f_type)[0]
        b_data = convert_to_binary(significant_b_data)
        auc_value_b = kf_average_2(b_data,c_method)
        auc_array_b_AST1.append(auc_value_b)
        
        
        
        b_data = select_features_1(data_3,i,score_f_type)[0]
        auc_value_p = kf_average_2(b_data,c_method)
        auc_array_p_AST2.append(auc_value_p)
        
        b_data = select_features_1(data_4,i,score_f_type)[0]
        auc_value_c = kf_average_2(b_data,c_method)
        auc_array_c_AST2.append(auc_value_c)
        
        significant_b_data = select_features_1(data_4,i,score_f_type)[0]
        b_data = convert_to_binary(significant_b_data)
        auc_value_b = kf_average_2(b_data,c_method)
        auc_array_b_AST2.append(auc_value_b)
        
        
        
        b_data = select_features_1(data_5,i,score_f_type)[0]
        auc_value_p = kf_average_2(b_data,c_method)
        auc_array_p_AST3.append(auc_value_p)
        
        b_data = select_features_1(data_6,i,score_f_type)[0]
        auc_value_c = kf_average_2(b_data,c_method)
        auc_array_c_AST3.append(auc_value_c)
        
        significant_b_data = select_features_1(data_6,i,score_f_type)[0]
        b_data = convert_to_binary(significant_b_data)
        auc_value_b = kf_average_2(b_data,c_method)
        auc_array_b_AST3.append(auc_value_b)
        
        
        
        b_data = select_features_1(data_7,i,score_f_type)[0]
        auc_value_p = kf_average_2(b_data,c_method)
        auc_array_p_AST4.append(auc_value_p)
        
        b_data = select_features_1(data_8,i,score_f_type)[0]
        auc_value_c = kf_average_2(b_data,c_method)
        auc_array_c_AST4.append(auc_value_c)
        
        significant_b_data = select_features_1(data_8,i,score_f_type)[0]
        b_data = convert_to_binary(significant_b_data)
        auc_value_b = kf_average_2(b_data,c_method)
        auc_array_b_AST4.append(auc_value_b)
        
    return(features_n,auc_array_p_AST1,auc_array_c_AST1,auc_array_b_AST1,auc_array_p_AST2,auc_array_c_AST2,auc_array_b_AST2,auc_array_p_AST3,auc_array_c_AST3,auc_array_b_AST3,auc_array_p_AST4,auc_array_c_AST4,auc_array_b_AST4)

In [ ]:
score_f_type = 4

In [ ]:
# Logistic Regression
result = compute_values(d_1_p,d_1_c,d_2_p,d_2_c,d_3_p,d_3_c,d_4_p,d_4_c,score_f_type,1)

In [ ]:
# Random Forest
m_depth = 20
result = compute_values(d_1_p,d_1_c,d_2_p,d_2_c,d_3_p,d_3_c,d_4_p,d_4_c,score_f_type,2)

In [ ]:
# KNN
result = compute_values(d_1_p,d_1_c,d_2_p,d_2_c,d_3_p,d_3_c,d_4_p,d_4_c,score_f_type,9)

In [ ]:
# Gaussian Naive Bayes
result = compute_values(d_1_p,d_1_c,d_2_p,d_2_c,d_3_p,d_3_c,d_4_p,d_4_c,score_f_type,10)

In [ ]:
# RFE Logistic Regression
result = compute_values_RFE(d_1_p,d_1_c,d_2_p,d_2_c,d_3_p,d_3_c,d_4_p,d_4_c,1)

In [ ]:
print(result)

In [ ]:
features_n,auc_array_p_AST1,auc_array_c_AST1,auc_array_b_AST1,auc_array_p_AST2,auc_array_c_AST2,auc_array_b_AST2,auc_array_p_AST3,auc_array_c_AST3,auc_array_b_AST3,auc_array_p_AST4,auc_array_c_AST4,auc_array_b_AST4 = result

In [ ]:
plt.rcParams["figure.figsize"] = (10,6)
 
plt.plot(features_n,auc_array_p_AST1,color='red',marker='o',label='Probability_AST1')
plt.plot(features_n,auc_array_c_AST1,color='red',marker='o',linestyle='--',dashes=(3,3),label='Count_AST1')
plt.plot(features_n,auc_array_b_AST1,color='red',marker='o',linestyle='dotted',label='Binary_AST1')

plt.plot(features_n,auc_array_p_AST2,color='black',marker='o',label='Probability_AST2')
plt.plot(features_n,auc_array_c_AST2,color='black',marker='o',linestyle='--',dashes=(3,3),label='Count_AST2')
plt.plot(features_n,auc_array_b_AST2,color='black',marker='o',linestyle='dotted',label='Binary_AST2')

plt.plot(features_n,auc_array_p_AST3,color='skyblue',marker='o',label='Probability_AST3')
plt.plot(features_n,auc_array_c_AST3,color='skyblue',marker='o',linestyle='--',dashes=(3,3),label='Count_AST3')
plt.plot(features_n,auc_array_b_AST3,color='skyblue',marker='o',linestyle='dotted',label='Binary_AST3')

plt.plot(features_n,auc_array_p_AST4,color='blue',marker='o',label='Probability_AST4')
plt.plot(features_n,auc_array_c_AST4,color='blue',marker='o',linestyle='--',dashes=(3,3),label='Count_AST4')
plt.plot(features_n,auc_array_b_AST4,color='blue',marker='o',linestyle='dotted',label='Binary_AST4')

plt.title('Diagram of ROC-AUC and number of features',fontsize=16)
plt.xlabel('Number of features',fontsize=14)
plt.ylabel('ROC-AUC',fontsize=14)
plt.legend(fontsize=11,loc='lower right')
plt.show()

In [ ]:
print(d_4_c.shape)

In [ ]:
d_1_p = np.load('./data_2/dense_trigram_data_AST1.npy')
d_1_c = np.load('./data_2/dense_trigram_data_c_AST1.npy')

d_2_p = np.load('./data_2/dense_trigram_data_AST2.npy')
d_2_c = np.load('./data_2/dense_trigram_data_c_AST2.npy')

d_3_p = np.load('./data_2/dense_trigram_data_AST3.npy')
d_3_c = np.load('./data_2/dense_trigram_data_c_AST3.npy')

d_4_p = np.load('./data_2/dense_trigram_data_AST4.npy')
d_4_c = np.load('./data_2/dense_trigram_data_c_AST4.npy')

d_5_p = np.load('./data_2/dense_trigram_data.npy')
d_5_c = np.load('./data_2/dense_trigram_data.npy')

# Selecting subset of data:

In [ ]:
def compute_values_RFE(data_1,data_2,data_3,data_4,data_5,data_6,data_7,data_8,data_9,data_10,c_method):
    d_1 =data_1.copy()
    d_2 =data_2.copy()
    d_3 =data_3.copy()
    d_4 =data_4.copy()
    d_5 =data_5.copy()
    d_6 =data_6.copy()
    d_7 =data_7.copy()
    d_8 =data_8.copy()
    d_9 =data_9.copy()
    d_10 =data_10.copy()
    
    features_n = []
    
    auc_array_p_AST1 = []
    auc_array_c_AST1 = []
    auc_array_b_AST1 = []
    
    auc_array_p_AST2 = []
    auc_array_c_AST2 = []
    auc_array_b_AST2 = []
    
    auc_array_p_AST3 = []
    auc_array_c_AST3 = []
    auc_array_b_AST3 = []
    
    auc_array_p_AST4 = []
    auc_array_c_AST4 = []
    auc_array_b_AST4 = []
    
    auc_array_p_word = []
    auc_array_c_word = []
    auc_array_b_word = []
    
    if (c_method == 1):
        score_f_type = 7
    elif (c_method == 2):
        score_f_type = 8
        
    for i in range(250,24,-25):
#     for i in range(150,49,-50):
        features_n.append(i)
        
        d_1 = select_features_2(d_1,i,score_f_type)[0]
        auc_value_p = kf_average_2(d_1,c_method)
        auc_array_p_AST1.append(auc_value_p)
        
        d_2 = select_features_2(d_2,i,score_f_type)[0]
        auc_value_c = kf_average_2(d_2,c_method)
        auc_array_c_AST1.append(auc_value_c)
        
        significant_b_data = select_features_2(d_2,i,score_f_type)[0]
        b_data = convert_to_binary(significant_b_data)
        auc_value_b = kf_average_2(b_data,c_method)
        auc_array_b_AST1.append(auc_value_b)
        
        
        
        d_3 = select_features_2(d_3,i,score_f_type)[0]
        auc_value_p = kf_average_2(d_3,c_method)
        auc_array_p_AST2.append(auc_value_p)
        
        d_4 = select_features_2(d_4,i,score_f_type)[0]
        auc_value_c = kf_average_2(d_4,c_method)
        auc_array_c_AST2.append(auc_value_c)
        
        b_data = convert_to_binary(d_4)
        auc_value_b = kf_average_2(b_data,c_method)
        auc_array_b_AST2.append(auc_value_b)
        
        
        
        d_5 = select_features_2(d_5,i,score_f_type)[0]
        auc_value_p = kf_average_2(d_5,c_method)
        auc_array_p_AST3.append(auc_value_p)
        
        d_6 = select_features_2(d_6,i,score_f_type)[0]
        auc_value_c = kf_average_2(d_6,c_method)
        auc_array_c_AST3.append(auc_value_c)
        
        b_data = convert_to_binary(d_6)
        auc_value_b = kf_average_2(b_data,c_method)
        auc_array_b_AST3.append(auc_value_b)
        
        
        
        print(d_7.shape)
        d_7 = select_features_2(d_7,i,score_f_type)[0]
        print(d_7.shape)
        auc_value_p = kf_average_2(d_7,c_method)
        auc_array_p_AST4.append(auc_value_p)
        
        d_8 = select_features_2(d_8,i,score_f_type)[0]
        auc_value_c = kf_average_2(d_8,c_method)
        auc_array_c_AST4.append(auc_value_c)
        
        b_data = convert_to_binary(d_8)
        auc_value_b = kf_average_2(b_data,c_method)
        auc_array_b_AST4.append(auc_value_b)
        
        
        
        d_9 = select_features_2(d_9,i,score_f_type)[0]
        auc_value_p = kf_average_2(d_9,c_method)
        auc_array_p_word.append(auc_value_p)
        
        d_10 = select_features_2(d_10,i,score_f_type)[0]
        auc_value_c = kf_average_2(d_10,c_method)
        auc_array_c_word.append(auc_value_c)
        
        b_data = convert_to_binary(d_10)
        auc_value_b = kf_average_2(b_data,c_method)
        auc_array_b_word.append(auc_value_b)
        
    return(features_n,auc_array_p_AST1,auc_array_c_AST1,auc_array_b_AST1,auc_array_p_AST2,auc_array_c_AST2,auc_array_b_AST2,auc_array_p_AST3,auc_array_c_AST3,auc_array_b_AST3,auc_array_p_AST4,auc_array_c_AST4,auc_array_b_AST4,auc_array_p_word,auc_array_c_word,auc_array_b_word)

In [ ]:
def select_data_subset(data_1,c):
    start_time = time.time()
    if (c == 1):
        new_d = select_features_2(data_1,400,7)[0]
    elif (c == 2):
        new_d = select_features_2(data_1,400,8)[0]
    finish_time = time.time()
    elapsed_time = finish_time - start_time
    print(elapsed_time)
    return new_d

In [ ]:
# 1:LR,   2:RF,   9:KNN,   10:GNB
c1 = 1

In [ ]:
step_value = 25
d_1_p_2 = select_data_subset(d_1_p,c1)
d_1_c_2 = select_data_subset(d_1_c,c1)

step_value = 25
d_2_p_2 = select_data_subset(d_2_p,c1)
d_2_c_2 = select_data_subset(d_2_c,c1)

step_value = 25
d_3_p_2 = select_data_subset(d_3_p,c1)
d_3_c_2 = select_data_subset(d_3_c,c1)

step_value = 25
d_4_p_2 = select_data_subset(d_4_p,c1)
d_4_c_2 = select_data_subset(d_4_c,c1)

step_value = 25
d_5_p_2 = select_data_subset(d_5_p,c1)
d_5_c_2 = select_data_subset(d_5_c,c1)

In [ ]:
step_value = 25
d_1_p_2 = select_data_subset(d_1_p,c1)
d_1_c_2 = select_data_subset(d_1_c,c1)

In [ ]:
step_value = 25
d_2_p_2 = select_data_subset(d_2_p,c1)
d_2_c_2 = select_data_subset(d_2_c,c1)

In [ ]:
step_value = 25
d_3_p_2 = select_data_subset(d_3_p,c1)
d_3_c_2 = select_data_subset(d_3_c,c1)

In [ ]:
step_value = 25
d_4_p_2 = select_data_subset(d_4_p,c1)
d_4_c_2 = select_data_subset(d_4_c,c1)

In [ ]:
step_value = 25
d_5_p_2 = select_data_subset(d_5_p,c1)
d_5_c_2 = select_data_subset(d_5_c,c1)

In [ ]:
# RFE Logistic Regression
step_value = 25
result = compute_values_RFE(d_1_p_2, d_1_c_2, d_2_p_2, d_2_c_2, d_3_p_2, d_3_c_2, d_4_p_2, d_4_c_2, d_5_p_2, d_5_c_2, 1)

In [ ]:
# RFE Random Forest
step_value = 25
m_depth = 10
result = compute_values_RFE(d_1_p_2, d_1_c_2, d_2_p_2, d_2_c_2, d_3_p_2, d_3_c_2, d_4_p_2, d_4_c_2, d_5_p_2, d_5_c_2, 2)

In [ ]:
features_n,auc_array_p_AST1,auc_array_c_AST1,auc_array_b_AST1,auc_array_p_AST2,auc_array_c_AST2,auc_array_b_AST2,auc_array_p_AST3,auc_array_c_AST3,auc_array_b_AST3,auc_array_p_AST4,auc_array_c_AST4,auc_array_b_AST4,auc_array_p_word,auc_array_c_word,auc_array_b_word = result
print(result)

In [ ]:
for r in result:
    print(np.max(r))

In [ ]:
plt.rcParams["figure.figsize"] = (10,6)
 
plt.plot(features_n,auc_array_p_AST1,color='red',marker='o',label='Probability_AST1')
plt.plot(features_n,auc_array_c_AST1,color='red',marker='o',linestyle='--',dashes=(3,3),label='Count_AST1')
plt.plot(features_n,auc_array_b_AST1,color='red',marker='o',linestyle='dotted',label='Binary_AST1')

plt.plot(features_n,auc_array_p_AST2,color='black',marker='o',label='Probability_AST2')
plt.plot(features_n,auc_array_c_AST2,color='black',marker='o',linestyle='--',dashes=(3,3),label='Count_AST2')
plt.plot(features_n,auc_array_b_AST2,color='black',marker='o',linestyle='dotted',label='Binary_AST2')

plt.plot(features_n,auc_array_p_AST3,color='skyblue',marker='o',label='Probability_AST3')
plt.plot(features_n,auc_array_c_AST3,color='skyblue',marker='o',linestyle='--',dashes=(3,3),label='Count_AST3')
plt.plot(features_n,auc_array_b_AST3,color='skyblue',marker='o',linestyle='dotted',label='Binary_AST3')

plt.plot(features_n,auc_array_p_AST4,color='blue',marker='o',label='Probability_AST4')
plt.plot(features_n,auc_array_c_AST4,color='blue',marker='o',linestyle='--',dashes=(3,3),label='Count_AST4')
plt.plot(features_n,auc_array_b_AST4,color='blue',marker='o',linestyle='dotted',label='Binary_AST4')

plt.plot(features_n,auc_array_p_word,color='grey',marker='o',label='Probability_word')
plt.plot(features_n,auc_array_c_word,color='grey',marker='o',linestyle='--',dashes=(3,3),label='Count_word')
plt.plot(features_n,auc_array_b_word,color='grey',marker='o',linestyle='dotted',label='Binary_word')

plt.title('Diagram of ROC-AUC and number of features',fontsize=16)
plt.xlabel('Number of features',fontsize=14)
plt.ylabel('ROC-AUC',fontsize=14)
# plt.legend(fontsize=11,loc='lower right')
plt.gca().legend(loc='center left', bbox_to_anchor=(1,0.5))
plt.show()

In [ ]:
np.save('./data_2/dense_trigram_data_AST1_LR.npy',d_1_p_2)
np.save('./data_2/dense_trigram_data_c_AST1_LR.npy',d_1_c_2)

np.save('./data_2/dense_trigram_data_AST2_LR.npy',d_2_p_2)
np.save('./data_2/dense_trigram_data_c_AST2_LR.npy',d_2_c_2)

np.save('./data_2/dense_trigram_data_AST3_LR.npy',d_3_p_2)
np.save('./data_2/dense_trigram_data_c_AST3_LR.npy',d_3_c_2)

np.save('./data_2/dense_trigram_data_AST4_LR.npy',d_4_p_2)
np.save('./data_2/dense_trigram_data_c_AST4_LR.npy',d_4_c_2)

np.save('./data_2/dense_trigram_data_word_LR.npy',d_5_p_2)
np.save('./data_2/dense_trigram_data_c_word_LR.npy',d_5_c_2)

In [ ]:
d_1_p_2 = np.load('./data_2/dense_trigram_data_AST1_LR.npy')
d_1_c_2 = np.load('./data_2/dense_trigram_data_c_AST1_LR.npy')

d_2_p_2 = np.load('./data_2/dense_trigram_data_AST2_LR.npy')
d_2_c_2 = np.load('./data_2/dense_trigram_data_c_AST2_LR.npy')

d_3_p_2 = np.load('./data_2/dense_trigram_data_AST3_LR.npy')
d_3_c_2 = np.load('./data_2/dense_trigram_data_c_AST3_LR.npy')

d_4_p_2 = np.load('./data_2/dense_trigram_data_AST4_LR.npy')
d_4_c_2 = np.load('./data_2/dense_trigram_data_c_AST4_LR.npy')

d_5_p_2 = np.load('./data_2/dense_trigram_data_word_LR.npy')
d_5_c_2 = np.load('./data_2/dense_trigram_data_c_word_LR.npy')

In [ ]:
print(d_5_p_2.shape)